<a href="https://colab.research.google.com/github/Vizzuality/sci_team_data_bank/blob/master/prepare_mangrove_atlas_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use this processing pipeline

Processed outputs should got to the various output options (GEE, MapBox, GCS, mangrove-atlas-API). Note some processes via GEE take some time, hence check via the JS code editor when they are ready. The idea is to have a completely reproducible pipeline, i.e, minimum processing done by hand.

1. Run the "set up environment" cells, these install packages, utilities, set-up permissions and define enviroment variables
1. Load the Python helper functions
1. Go to the data processing section and run the code! These should be independant, self-contained sections.  

### TODO

+ Streamline and refactor file conversions, simplification, and combination
+ Include any GEE JS code into this main pipeline
+ Look into how the process can be made to check for completion of asset uploads to GEE
+ Make an overview diagram showing data dependencies
 

#  Set up environment

+ install and authorise earthengine client
+ setup gcloud
+ install juypter node extension

In [0]:
# Sometimes changing settings and installed software does not work
# seems a full restart is needed, run this to kill the session then after 30 s refresh tab
# https://stackoverflow.com/questions/49001921/how-to-restart-virtual-machine
#!kill -9 -1

In [0]:
%%bash
# Install AWS CLI (for MapBox uploads)
sudo apt install --no-install-recommends -y -q awscli

In [0]:
%%bash
# Install tippecanoe (for MapBox mbtiles)
sudo apt install --no-install-recommends -q -y build-essential libsqlite3-dev zlib1g-dev
make
make install
sudo add-apt-repository -y ppa:ubuntu-toolchain-r/test
sudo apt update -q -y
sudo apt install --no-install-recommends -q -y g++-5
export CXX=g++-5

git clone https://github.com/mapbox/tippecanoe.git
cd tippecanoe
make -j
make install

In [0]:
!tippecanoe -v

tippecanoe v1.34.5


In [0]:
%%bash
# Install python packages (google api, gee api)
sudo pip install google-api-python-client earthengine-api mapbox

In [0]:
%%bash
# Install mapshaper CLI (simplification, combining, change format)
npm install -g mapshaper

/tools/node/bin/mapshaper -> /tools/node/lib/node_modules/mapshaper/bin/mapshaper
/tools/node/bin/mapshaper-gui -> /tools/node/lib/node_modules/mapshaper/bin/mapshaper-gui
/tools/node/bin/mapshaper-xl -> /tools/node/lib/node_modules/mapshaper/bin/mapshaper-xl
+ mapshaper@0.4.137
added 50 packages from 71 contributors in 5.475s


In [0]:
# Set up Mapbox (S3) credentials as environmental variables
import json

%env MB_USER=mangrove-atlas
%env MB_TOKEN=sk.eyJ1IjoibWFuZ3JvdmUtYXRsYXMiLCJhIjoiY2p3MzlxZjZzMGdsZjRicXJqMGM2d2F4dSJ9.BUecVt2pYITX3DXF6N4_0w
!curl -X POST https://api.mapbox.com/uploads/v1/${MB_USER}/credentials?access_token=${MB_TOKEN} > credentials.json
r = json.loads(open("credentials.json").read())
import os
os.environ['MB_BUCKET'] = r['bucket']
os.environ['MB_KEY'] = r['key']
os.environ['AWS_ACCESS_KEY_ID'] = r['accessKeyId']
os.environ['AWS_SECRET_ACCESS_KEY'] = r['secretAccessKey']
os.environ['AWS_SESSION_TOKEN'] = r['sessionToken']

env: MB_USER=mangrove-atlas
env: MB_TOKEN=sk.eyJ1IjoibWFuZ3JvdmUtYXRsYXMiLCJhIjoiY2p3MzlxZjZzMGdsZjRicXJqMGM2d2F4dSJ9.BUecVt2pYITX3DXF6N4_0w
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   829  100   829    0     0   2431      0 --:--:-- --:--:-- --:--:--  2431


In [0]:
# Set some useful environment variables
# GCS bucket and local base directory
import os
os.environ['MA_BUCKET'] = 'mangrove_atlas'
# Mapshaper variables for simplification
os.environ['SIMP_WEIGHT_INT'] = '30'
os.environ['MIN_AREA'] = '90'

In [0]:
%%bash
# remove sample data and make local base dir
rm -rf /var/lib/apt/lists/*
rm -r sample_data $MA_BUCKET
mkdir -p $MA_BUCKET

rm: cannot remove 'mangrove_atlas': No such file or directory


## Authenticate

In [0]:
# Setup Google Cloud Storage
# Note this is a personal, public bucket owned by Lamert (Wetlands International)
# Google cloud storage set project
project_id = 'mangrove-atlas-246414'
!gcloud config set project {project_id}

# Authenticate
from google.colab import auth
auth.authenticate_user()

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud alpha survey

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
# Import the Earth Engine library.
import ee

# Trigger the authentication flow.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/sAExlhsmGkfqAfq5VPci13YKE2MhrAgn1MJssmMntv2HKqd5dbveZ0E

Successfully saved authorization token.


In [0]:
!ls 

adc.json	  mangrove_atlas	     tippecanoe
credentials.json  test_upload_manifest.json


# Load helper functions

In [0]:
# Python helper functions

# Upload task for mapbox
import os
import subprocess
import sys
from mapbox import Uploader
import base64
from time import sleep
from random import randint


def upload_to_mapbox(file_path, tileset_name):
  '''Given a local file path and a Map Box tileset name
     Push to Map Box AWS S3 staging and create Map Box upload task
  '''
  username = os.getenv("MB_USER")
  MY_TOKEN = os.getenv("MB_TOKEN")
  #print(MY_TOKEN)
  #MY_TOKEN = base64.b64decode(MY_TOKEN + "===")
  #conxn = Uploader(username, MY_TOKEN)
  #resp = conxn.upload(file_path, tileset_name)
  #upload_id = resp.json()['id']

  #resp = conxn.status(upload_id).json()
  #print("Upload {0} with id {1}".format(resp['complete'], resp['tileset']))
  u = Uploader(access_token=MY_TOKEN)  # handles authentication
  tileset = username + "." + tileset_name  # name your tileset
  url = u.stage(open(file_path, 'rb'))  # upload happens here
  job = u.create(url, tileset, name=tileset_name)  # starts the tiling job
  status = u.status(job).json()
  print(status)
  service = Uploader()
mapid = getfixture('uploads_dest_id') # 'uploads-test'
with open('tests/twopoints.geojson', 'rb') as src:
     upload_resp = service.upload(src, mapid)

 if upload_resp.status_code == 422:
     for i in range(5):
         sleep(5)
         with open('tests/twopoints.geojson', 'rb') as src:
             upload_resp = service.upload(src, mapid)
         if upload_resp.status_code != 422:
             break

  
  
  # Upload to mapbox S3 staging
  #MB_BUCKET = os.getenv("MB_BUCKET")
  #MB_KEY = os.getenv("MB_KEY")
  #cmd = "aws s3 cp " + file_path + " s3://" + MB_BUCKET + "/" + MB_KEY + " --region us-east-1"
  #print(cmd)
  #try:
  #  retcode = subprocess.call(cmd, shell=True)
  #  if retcode < 0:
  #      print("Child was terminated by signal", -retcode, file=sys.stderr)
  #  else:
  #      print("Child returned", retcode, file=sys.stderr)
  #except OSError as e:
  #  print("Execution failed:", e, file=sys.stderr)
  # Make mapbox upload task
  #MB_USER = os.getenv("MB_USER")
  #MB_TOKEN = os.getenv("MB_TOKEN")
  #d = "'{'url': 'http://" + MB_BUCKET + ".s3.amazonaws.com" + MB_KEY + "', 'tileset': '" + MB_USER + "." + tileset_name + "', 'name': '" + tileset_name + "'}'"
  #u = "https://api.mapbox.com/uploads/v1/" + MB_USER + "?access_token=" + MB_TOKEN
  #cmd2 = "curl -X POST -H 'Content-Type: application/json' -H 'Cache-Control: no-cache' -d " + d + " " + u
  #print(cmd2)
  #try:
  #  retcode = subprocess.call(cmd2, shell=True)
  #  if retcode < 0:
  #      print("Child was terminated by signal", -retcode, file=sys.stderr)
  #  else:
  #      print("Child returned", retcode, file=sys.stderr)
  #except OSError as e:
  #  print("Execution failed:", e, file=sys.stderr)
       
# Get resolution info from given raster
def get_pixelsize(file_path):
    cmd = "( $(gdalinfo ", + file_path + " | grep 'Pixel Size' | sed 's/Pixel Size =//g; s/,/ /g' | tr '\n' ' ' | tr -d '[(,])-') )"
    print(cmd)
    try:
      retcode = subprocess.check_output(cmd, shell=True).decode('utf8')
      if retcode < 0:
        print("Child was terminated by signal", -retcode, file=sys.stderr)
      else:
        print("Child returned", retcode, file=sys.stderr)
    except OSError as e:
      print("Execution failed:", e, file=sys.stderr)
    return retcode

def get_navalue(file_path):
    cmd = "( $(gdalinfo ", + file_path + " | grep 'NoData Value' | sed 's/NoData Value =//g; s/,/ /g' | tr '\n' ' ' | tr -d '[(,])') )"
    print(cmd)
    try:
      retcode = subprocess.check_output(cmd, shell=True).decode('utf8')
      if retcode < 0:
        print("Child was terminated by signal", -retcode, file=sys.stderr)
      else:
        print("Child returned", retcode, file=sys.stderr)
    except OSError as e:
      print("Execution failed:", e, file=sys.stderr)
    return retcode

  
def gee_upload_images_to_collection(file_path, asset_path, pyramiding_policy, bands, time_start, nodata_value, force=True, properties={}):
    '''Given a GCS file path and a GEE asset path
       Create, and push to all images + metadata to a SINGLE ee.ImageCollection asset
    '''
    # Format arguments
    f = ""
    if force:
      f = "--force"
    pp = "--pyramiding_policy={0}".format(pyramiding_policy) 
    ts = "--time_start={0}".format(time_start)
    n = "--nodata_value={0}".format(nodata_value)
    b =  "--bands={0}".format(bands)
    p = ""
    if len(properties) > 0:
      p = ["--property={0}={1}".format(key, value) for key, value in properties.items()]
      p = " ".join(p) 
    args = "{0} {1} {2} {3} {4} {5}".format(f, pp, ts, n, b, p)
    # Get file path array
    cmd = "gsutil ls " + file_path
    file_array = subprocess.check_output(cmd, shell=True).decode('utf8').split('\n')
    print("Found {0} files".format(len(file_array)))
    # Create collection
    cmd = "earthengine --no-use_cloud_api create collection {0}".format(asset_path)
    r = subprocess.call(cmd, shell=True)
    # Create upload task for each file
    for file in file_array:
      asset_id = os.path.splitext(os.path.basename(file))[0]
      print("Processing {0}".format(asset_id))
      cmd = "earthengine --no-use_cloud_api upload image --asset_id={0}/{1} {2} {3}".format(asset_path, asset_id, args, file)
      print(cmd)
      r = subprocess.call(cmd, shell=True)
      if r == 0:
        print("Task created")
      else:
        print("Task failed")
    print("Finished upload")   

def gee_upload_single_image(file_path, asset_path, pyramiding_policy, bands, time_start, nodata_value, force=True, properties={}):
    '''Given a GCS file path and a GEE asset path
       Create, and push to all images + metadata to a SINGLE ee.Image asset
    '''
    # Format arguments
    f = ""
    if force:
      f = "--force"
    pp = "--pyramiding_policy={0}".format(pyramiding_policy) 
    ts = "--time_start={0}".format(time_start)
    n = "--nodata_value={0}".format(nodata_value)
    b =  "--bands={0}".format(bands)
    p = ""
    if len(properties) > 0:
      p = ["--property={0}={1}".format(key, value) for key, value in properties.items()]
      p = " ".join(p) 
    args = "{0} {1} {2} {3} {4} {5}".format(f, pp, ts, n, b, p)
    # Get file path array
    cmd = "gsutil ls " + file_path
    file_array = subprocess.check_output(cmd, shell=True).decode('utf8').split('\n')
    print("Found {0} files".format(len(file_array)))
    # Create upload task for all files
    print("Processing {0}".format(asset_path))
    file_string = " ".join(file_array)
    cmd = "earthengine --no-use_cloud_api upload image --asset_id={0} {1} {2}".format(asset_path, args, file_string)
    print(cmd)
    r = subprocess.call(cmd, shell=True)
    if r == 0:
      print("Task created")
    else:
      print("Task failed")
    print("Finished upload")
    
def gee_upload_images_to_image(file_path, asset_path, pyramiding_policy, bands, time_start, nodata_value, force=True, properties={}):
    '''Given a GCS file path and a GEE asset path
       Create, and push to all images + metadata to a SINGLE ee.Image asset
    '''
    # Format arguments
    f = ""
    if force:
      f = "--force"
    pp = "--pyramiding_policy={0}".format(pyramiding_policy) 
    ts = "--time_start={0}".format(time_start)
    n = "--nodata_value={0}".format(nodata_value)
    b =  "--bands={0}".format(bands)
    p = ""
    if len(properties) > 0:
      p = ["--property={0}={1}".format(key, value) for key, value in properties.items()]
      p = " ".join(p) 
    args = "{0} {1} {2} {3} {4} {5}".format(f, pp, ts, n, b, p)
    # Get file path array
    cmd = "gsutil ls " + file_path
    file_array = subprocess.check_output(cmd, shell=True).decode('utf8').split('\n')
    with open('tmp.txt', mode='wt', encoding='utf-8') as myfile:
      myfile.write('\n'.join(file_array))
    print("Found {0} files".format(len(file_array)))
    # Create upload task for all files
    print("Processing {0}".format(asset_path))
    #" ".join(file_array)
    cmd = "earthengine --no-use_cloud_api upload image --asset_id={0} {1} < tmp.txt".format(asset_path, args)
    print(cmd)
    r = subprocess.call(cmd, shell=True)
    if r == 0:
      print("Task created")
    else:
      print("Task failed")
    print("Finished upload")
        
def gee_upload_zips_to_tables(file_path, asset_path, time_start, force=True, append_txt="", properties={}):
    '''Given a GCS file path and a GEE asset path
       Create, and push to all zipped shapefiles + metadata to MANY Table assets
    '''
    # Format arguments
    f = ""
    if force:
      f = "--force"
    ts = "--time_start={0}".format(time_start)
    p = ""
    if len(properties) > 0:
      p = ["--property='{0}={1}'".format(key, value) for key, value in properties.items()]
      p = " ".join(p) 
    args = "{0} {1} {2} ".format(f, ts, p)
    # Get file path array
    cmd = "gsutil ls " + file_path
    file_array = subprocess.check_output(cmd, shell=True).decode('utf8').split('\n')
    print("Found {0} files".format(len(file_array)))
    # Create table and upload task for each file
    for file in file_array:
      asset_id = os.path.splitext(os.path.basename(file))[0]
      print("Processing {0}".format(asset_id))
      # Create collection
      #cmd = "earthengine create collection {0}/{1}".format(asset_path, asset_id)
      #r = subprocess.call(cmd, shell=True)
      cmd = "earthengine --no-use_cloud_api upload table --asset_id={0}/{1}{2} {3} {4}".format(asset_path, asset_id, append_txt, args, file)
      print(cmd)
      r = subprocess.call(cmd, shell=True)
      if r == 0:
        print("Task created")
      else:
        print("Task failed")
    print("Finished upload")    
    



---


# Data description

+ all data is stored in the public bucket `mangrove_atlas`, from project `mangrove-atlas-246414`
    + https://console.cloud.google.com/storage/browser?project=mangrove-atlas-246414
+ directory names _try_ to use the controlled vocabulary https://www.eionet.europa.eu/gemet/en/

## General directory structure

`mangrove_atlas/`
 
 + `ee-export-tables` - directory for storing output from GEE analysis
 + `ee-upload-mainfest` - directory for storing GEE upload manifests
 + `boundaries` - adminstatrive, proctected areas, areas-of-interest (AOIs)
 + `land-cover` - mangrove extent
 + `mangrove-properties` - properties (biomass, height, ect.) of mangrove land-cover
 + `orthoimagery` - georeferenced imagery from satellites and airbourne sensors (e.g., UAVs)
 + `environmental-pressures` - layers related to drivers and pressures on mangrove habitats
 + `physical-environment` - layers relating to coast lines, tides, hydrodynamics, freshwater availability, and sediments
 + `elevation` - digital elevation, surface, and terrain models (DEM, DSM, and DTM)

In [0]:
# To avoid disk usage issues get in sections and clean up afterwards
#!gsutil -m cp -r gs://ma-data .

# Process boundaries

+ Add zipped boundaries shapefiles to GCS bucket
+ updated WDPA datset is available in earthengine "WCMC/WDPA/current/polygons"
+ Clean and simplify
+ Add back to 'processed/' in bucket

In [0]:
%%bash
# Get boundaries data
gsutil -m cp -r gs://$MA_BUCKET/boundaries $MA_BUCKET

Copying gs://mangrove_atlas/boundaries/aoi_rufiji_TZA.zip...
Copying gs://mangrove_atlas/boundaries/EEZ_land_union_v2_201410.zip...
Copying gs://mangrove_atlas/boundaries/processed/geojson/aoi_saloum_SEN.json...
Copying gs://mangrove_atlas/boundaries/processed/aoi_rufiji_TZA.zip...
Copying gs://mangrove_atlas/boundaries/aoi_saloum_SEN.zip...
Copying gs://mangrove_atlas/boundaries/processed/geojson/EEZ_land_v2_201410.json...
Copying gs://mangrove_atlas/boundaries/processed/geojson/selected_EEZ_land_v2_201410.json...
Copying gs://mangrove_atlas/boundaries/processed/aoi_saloum_SEN.zip...
Copying gs://mangrove_atlas/boundaries/processed/EEZ_land_v2_201410.zip...
Copying gs://mangrove_atlas/boundaries/processed/geojson/aoi_rufiji_TZA.json...
Copying gs://mangrove_atlas/boundaries/processed/geojson/selected_EEZ_land_v2_201410_simplifiedee_export.geojson...
Copying gs://mangrove_atlas/boundaries/processed/geojson/selected_WDPA_polygons.json...
Copying gs://mangrove_atlas/boundaries/processed/

In [0]:
ls $MA_BUCKET/boundaries

aoi_rufiji_TZA.zip  EEZ_land_union_v2_201410.zip
aoi_saloum_SEN.zip  processed/


In [0]:
%%bash
# Process zipped shapefiles
# Clean and simplfy (parameters set above in section setup)
# Convert to geojson
# change mapshaper settings below, see https://github.com/mbloch/mapshaper/wiki/Command-Reference

# Set working directories
df=$MA_BUCKET/boundaries
sf=$MA_BUCKET/boundaries/shapefiles
pf=$MA_BUCKET/boundaries/processed

# Create directories (if needed)
mkdir $sf
mkdir $pf
mkdir ${pf}/geojson

# Extract original ZIP to $sf
arr=($df/*.zip)
for ((i=0; i<${#arr[@]}; i++)); do
  z="${arr[$i]}"
  echo $z
  unzip -o $z -d $sf
  f="${z%%.*}"
done

# Clean and simplify original SHP
s=$sf/*.shp
echo $s
mapshaper-xl \
  -i snap files="${s}" \
  -simplify weighted interval=$SIMP_WEIGHT_INT \
  -clean min-gap-area=$MIN_AREA \
  -filter-islands min-area=$MIN_AREA \
  -filter-slivers min-area=$MIN_AREA \
  -clean min-gap-area=$MIN_AREA \
  -o $pf format=shapefile precision=0.000001 force

# Export processed SHP as geojson to $pf/geojson
sp=$pf/*.shp
echo $sp
mapshaper-xl \
  -i files="${sp}" \
  -o ${pf}/geojson format=geojson precision=0.000001 force

mangrove_atlas/boundaries/aoi_rufiji_TZA.zip
Archive:  mangrove_atlas/boundaries/aoi_rufiji_TZA.zip
 extracting: mangrove_atlas/boundaries/shapefiles/aoi_rufiji_TZA.cpg  
  inflating: mangrove_atlas/boundaries/shapefiles/aoi_rufiji_TZA.dbf  
  inflating: mangrove_atlas/boundaries/shapefiles/aoi_rufiji_TZA.prj  
  inflating: mangrove_atlas/boundaries/shapefiles/aoi_rufiji_TZA.shp  
  inflating: mangrove_atlas/boundaries/shapefiles/aoi_rufiji_TZA.shx  
mangrove_atlas/boundaries/aoi_saloum_SEN.zip
Archive:  mangrove_atlas/boundaries/aoi_saloum_SEN.zip
  inflating: mangrove_atlas/boundaries/shapefiles/aoi_saloum_SEN.dbf  
  inflating: mangrove_atlas/boundaries/shapefiles/aoi_saloum_SEN.prj  
  inflating: mangrove_atlas/boundaries/shapefiles/aoi_saloum_SEN.qpj  
  inflating: mangrove_atlas/boundaries/shapefiles/aoi_saloum_SEN.shp  
  inflating: mangrove_atlas/boundaries/shapefiles/aoi_saloum_SEN.shx  
mangrove_atlas/boundaries/EEZ_land_union_v2_201410.zip
Archive:  mangrove_atlas/boundaries

mkdir: cannot create directory ‘mangrove_atlas/boundaries/processed’: File exists
mkdir: cannot create directory ‘mangrove_atlas/boundaries/processed/geojson’: File exists
Allocating 8000 megabytes of heap memory
[i] Snapped 1149 points
[clean] Detected DBF text encoding: win1252
Sample text containing non-ascii characters:  Curaçao
[clean] Retained 260 of 261 features
[filter-islands] Removed 0 islands
[filter-slivers] Removed 0 slivers
[clean] Retained 260 of 260 features
[o] Wrote mangrove_atlas/boundaries/processed/EEZ_land_v2_201410.shp
[o] Wrote mangrove_atlas/boundaries/processed/EEZ_land_v2_201410.shx
[o] Wrote mangrove_atlas/boundaries/processed/EEZ_land_v2_201410.dbf
[o] Wrote mangrove_atlas/boundaries/processed/EEZ_land_v2_201410.prj
[i] Snapped 1 point
[clean] Retained 1 of 1 features
[filter-islands] Removed 0 islands
[filter-slivers] Removed 0 slivers
[clean] Retained 1 of 1 features
[o] Wrote mangrove_atlas/boundaries/processed/aoi_rufiji_TZA.shp
[o] Wrote mangrove_atlas

In [0]:
!ls $MA_BUCKET/boundaries/processed
!ls $MA_BUCKET/boundaries/processed/geojson

aoi_rufiji_TZA.dbf  aoi_saloum_SEN.prj	    EEZ_land_v2_201410.shp
aoi_rufiji_TZA.prj  aoi_saloum_SEN.shp	    EEZ_land_v2_201410.shx
aoi_rufiji_TZA.shp  aoi_saloum_SEN.shx	    EEZ_land_v2_201410.zip
aoi_rufiji_TZA.shx  aoi_saloum_SEN.zip	    geojson
aoi_rufiji_TZA.zip  EEZ_land_v2_201410.dbf
aoi_saloum_SEN.dbf  EEZ_land_v2_201410.prj
aoi_rufiji_TZA.json
aoi_saloum_SEN.json
EEZ_land_v2_201410.json
selected_EEZ_land_v2_201410.json
selected_EEZ_land_v2_201410_simplifiedee_export.geojson
selected_WDPA_polygons_ee_export.geojson
selected_WDPA_polygons.json


In [0]:
# Zip processed shape files
%%bash
pf=$MA_BUCKET/boundaries/processed
fl=(${pf}/*.shp)
for ((i=0; i<${#fl[@]}; i++)); do
  z="${fl[$i]}"
  bn="${z%%.*}"
  zip $bn.zip $bn.*
  
done

updating: mangrove_atlas/boundaries/processed/aoi_rufiji_TZA.dbf (deflated 61%)
updating: mangrove_atlas/boundaries/processed/aoi_rufiji_TZA.prj (deflated 15%)
updating: mangrove_atlas/boundaries/processed/aoi_rufiji_TZA.shp (deflated 15%)
updating: mangrove_atlas/boundaries/processed/aoi_rufiji_TZA.shx (deflated 45%)
updating: mangrove_atlas/boundaries/processed/aoi_saloum_SEN.dbf (deflated 62%)
updating: mangrove_atlas/boundaries/processed/aoi_saloum_SEN.prj (deflated 15%)
updating: mangrove_atlas/boundaries/processed/aoi_saloum_SEN.shp (deflated 67%)
updating: mangrove_atlas/boundaries/processed/aoi_saloum_SEN.shx (deflated 47%)
updating: mangrove_atlas/boundaries/processed/EEZ_land_v2_201410.dbf (deflated 65%)
updating: mangrove_atlas/boundaries/processed/EEZ_land_v2_201410.prj (deflated 17%)
updating: mangrove_atlas/boundaries/processed/EEZ_land_v2_201410.shp (deflated 25%)
updating: mangrove_atlas/boundaries/processed/EEZ_land_v2_201410.shx (deflated 29%)


In [0]:
%%bash
# Upload to GCS bucket
pf=$MA_BUCKET/boundaries/processed
gsutil -m cp ${pf}/*.zip gs://${pf}/
gsutil -m cp -r ${pf}/geojson gs://${pf}/

Copying file://mangrove_atlas/boundaries/processed/aoi_rufiji_TZA.zip [Content-Type=application/zip]...
Copying file://mangrove_atlas/boundaries/processed/aoi_saloum_SEN.zip [Content-Type=application/zip]...
Copying file://mangrove_atlas/boundaries/processed/EEZ_land_v2_201410.zip [Content-Type=application/zip]...
-
Operation completed over 3 objects/1.6 MiB.                                      
Copying file://mangrove_atlas/boundaries/processed/geojson/aoi_rufiji_TZA.json [Content-Type=application/json]...
Copying file://mangrove_atlas/boundaries/processed/geojson/selected_WDPA_polygons_ee_export.geojson [Content-Type=application/octet-stream]...
Copying file://mangrove_atlas/boundaries/processed/geojson/selected_EEZ_land_v2_201410.json [Content-Type=application/json]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" v

In [0]:
# Add zipped SHPs as GEE assets
# FIXME use the manifest upload method?
# add METADATA!!
import os
file_path="gs://{}/boundaries/processed/*.zip".format(os.environ.get('MA_BUCKET'))
asset_path="projects/global-mangrove-watch/boundaries"
time_start="2010-01-01T00:00:00"

# Remove previous assets
!earthengine rm projects/global-mangrove-watch/boundaries/EEZ_land_v2_201410_simplified
!earthengine rm projects/global-mangrove-watch/boundaries/aoi_rufiji_TZA_simplified
!earthengine rm projects/global-mangrove-watch/boundaries/aoi_saloum_SEN_simplified

# Upload latest assets
gee_upload_zips_to_tables(file_path, asset_path, time_start, force=True, append_txt="_simplified", properties={})

Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

Found 4 files
Processing EEZ_land_v2_201410
earthengine --no-use_cloud_api upload table --asset_id=projects/global-mangrove-watch/boundaries/EEZ_land_v2_201410_simplified --force --time_start=2010-01-01T00:00:00   gs://mangrove_atlas/boundaries/processed/EEZ_land_v2_201410.zip
Task created
Processing aoi_rufiji_TZA
earthengine --no-use_cloud_api upload table --asset_id=projects/global-mangrove-watch/boundaries/aoi_rufiji_TZA_simplified --force --time_start=2010-01-01T00:00:00   gs://mangrove_atlas/boundaries/processed/aoi_rufiji_TZA.zip
Task created
Processing aoi_saloum_SEN
earthengine --no-use_cloud_api upload table --asset_id=projects/global-mangrove-watch/boundaries/aoi_saloum_SEN_simplified --force --time_start=2010-01-01T00:00

**Wait for the uploads to finish (check in code editor)**

In [0]:
# Filter EEZ_land to only include geometries with mangrove data
import ee
ee.Initialize()

# Load datasets
mr = ee.FeatureCollection("projects/global-mangrove-watch/boundaries/EEZ_land_v2_201410_simplified")
mg_2016 = ee.FeatureCollection("projects/global-mangrove-watch/land-cover/GMW_2016_v2_simplified")

# Define general bounding box of Mangrove data
mg_bbox = ee.Geometry.Polygon([[[-180, 33],[-180, -34],[180, -34],[180, 33]]], None, False);

# Filter featureCollection by the presence of Mangroves
mr = mr.filterMetadata("ISO_3digit", "not_equals", "ATA").filterBounds(mg_bbox)

# Check if regions intersect with mangrove data
mg = mg_2016.union()

def check_mg(f):
  g = f.geometry()
  m = mg_2016.filterBounds(g).union().first()
  out = f.copyProperties(f).set({'hasMG': f.intersects(m)})
  return out  

mr_sel = mr.map(check_mg)
mr_sel = mr_sel.filterMetadata("hasMG", "equals", True)

# Export to GEE Asset
!earthengine --no-use_cloud_api rm -r "projects/global-mangrove-watch/boundaries/selected_EEZ_land_v2_201410_simplified"
mytask = ee.batch.Export.table.toAsset(collection = mr_sel, description="asset-export-ma-countries", assetId="projects/global-mangrove-watch/boundaries/selected_EEZ_land_v2_201410_simplified");
mytask.start()

# Export to GCS bucket
mytask2 = ee.batch.Export.table.toCloudStorage(bucket = os.environ.get('MA_BUCKET'), collection = mr_sel, description = "GCS-export-ma-countries", fileFormat="GEOJSON", fileNamePrefix= "boundaries/processed/geojson/selected_EEZ_land_v2_201410_simplified_");
mytask2.start()

Asset does not exist or is not accessible: projects/global-mangrove-watch/boundaries/selected_EEZ_land_v2_201410_simplified


In [0]:
# Filter WDPA to only include geometries with mangrove data
import ee
ee.Initialize()

# Load datasets
wdpa = ee.FeatureCollection("projects/global-mangrove-watch/boundaries/wdpa_polygons_latest_cleaned")
mrs = ee.FeatureCollection("projects/global-mangrove-watch/boundaries/selected_EEZ_land_v2_201410_simplified")
mg_2016 = ee.FeatureCollection("projects/global-mangrove-watch/land-cover/GMW_2016_v2_simplified")

# Define general bounding box of Mangrove data
mg_bbox = ee.Geometry.Polygon([[[-180, 33],[-180, -34],[180, -34],[180, 33]]], None, False);

# Filter featureCollection by the presence of Mangroves
wdpa = wdpa.filterBounds(mg_bbox)

# Filter featureCollection by MARINE (not 0)
wdpa = wdpa.filterMetadata("MARINE", "not_equals", "0")

# WDPA CONTAINS MULTIPLE GEOMETRIES, A MIX OF LINE STRINGS AND POLYGONS
# WDPA IS NOW PREFILTERED
# FOR NOW ONLY FEATURES WITH ALL POLYGONS ARE SELECTED
#def set_geo_type(f):
#  return f.set('geo_type', f.geometry().type())
#wdpa = wdpa.map(set_geo_type).filter(ee.Filter.eq('geo_type', 'Polygon'))

# Get list of ISO codes with mangrove data
iso = ee.List(mrs.aggregate_array("ISO_3digit")).getInfo()

# Filter wdpa
out = wdpa.filter(ee.Filter.inList(opt_leftField="ISO3", opt_rightValue=iso))

# Export to GEE
!earthengine --no-use_cloud_api rm -r "projects/global-mangrove-watch/boundaries/selected_WDPA_polygons"
mytask = ee.batch.Export.table.toAsset(collection=out, description="export-ma-wdpa",assetId="projects/global-mangrove-watch/boundaries/selected_WDPA_polygons_")
mytask.start()

# Export to GCS bucket
mytask2 = ee.batch.Export.table.toCloudStorage(bucket = os.environ.get('MA_BUCKET'), collection = out, description = "GCS-export-ma-wpda", fileFormat="GEOJSON", fileNamePrefix= "boundaries/processed/geojson/selected_WDPA_polygons_");
mytask2.start()


Asset does not exist or is not accessible: projects/global-mangrove-watch/boundaries/selected_WDPA_polygons


In [0]:
%%bash
# Ensure GEE exported geojson are correct format
# Get processed boundaries data
gsutil -m cp -r gs://$MA_BUCKET/boundaries/processed/geojson $MA_BUCKET/boundaries/processed
ls $MA_BUCKET/boundaries/processed/geojson

mapshaper-xl \
  -i files=$MA_BUCKET/boundaries/processed/geojson/selected_EEZ_land_v2_201410_simplified_ee_export.geojson \
  -o $MA_BUCKET/boundaries/processed/geojson/selected_EEZ_land_v2_201410.json format=geojson precision=0.000001 force

mapshaper-xl \
  -i files=$MA_BUCKET/boundaries/processed/geojson/selected_WDPA_polygons_ee_export.geojson \
  -o $MA_BUCKET/boundaries/processed/geojson/selected_WDPA_polygons.json format=geojson precision=0.000001 force


aoi_rufiji_TZA.json
aoi_saloum_SEN.json
EEZ_land_v2_201410.json
selected_EEZ_land_v2_201410.json
selected_EEZ_land_v2_201410_simplifiedee_export.geojson
selected_WDPA_polygons_ee_export.geojson
selected_WDPA_polygons.json


Copying gs://mangrove_atlas/boundaries/processed/geojson/EEZ_land_v2_201410.json...
Copying gs://mangrove_atlas/boundaries/processed/geojson/aoi_saloum_SEN.json...
Copying gs://mangrove_atlas/boundaries/processed/geojson/aoi_rufiji_TZA.json...
Copying gs://mangrove_atlas/boundaries/processed/geojson/selected_WDPA_polygons.json...
Copying gs://mangrove_atlas/boundaries/processed/geojson/selected_EEZ_land_v2_201410.json...
Copying gs://mangrove_atlas/boundaries/processed/geojson/selected_WDPA_polygons_ee_export.geojson...
Copying gs://mangrove_atlas/boundaries/processed/geojson/selected_EEZ_land_v2_201410_simplifiedee_export.geojson...
/ [7/7 files][461.7 MiB/461.7 MiB] 100% Done                                    
Operation completed over 7 objects/461.7 MiB.                                    
Allocating 8000 megabytes of heap memory
[o] Wrote mangrove_atlas/boundaries/processed/geojson/selected_EEZ_land_v2_201410.json
Allocating 8000 megabytes of heap memory
[i] Error: File not found 

In [0]:
%%bash
# Make mapbox tiles
tippecanoe -zg -o mangrove_atlas/boundaries/processed/geojson/selected_WDPA_polygons.mbtiles --drop-densest-as-needed --extend-zooms-if-still-dropping mangrove_atlas/boundaries/processed/geojson/selected_WDPA_polygons.json
tippecanoe -zg -o mangrove_atlas/boundaries/processed/geojson/selected_EEZ_land_v2_201410.mbtiles --drop-densest-as-needed --extend-zooms-if-still-dropping mangrove_atlas/boundaries/processed/geojson/selected_EEZ_land_v2_201410.json


For layer 0, using name "selected_WDPA_polygons"
2278 features, 7651288 bytes of geometry, 301242 bytes of separate metadata, 274545 bytes of string pool
Choosing a maxzoom of -z2 for features about 120711 feet (36793 meters) apart
Choosing a maxzoom of -z11 for resolution of about 164 feet (50 meters) within features
  99.9%  11/1992/1152  
For layer 0, using name "selected_EEZ_land_v2_201410"
102 features, 704721 bytes of geometry, 2368 bytes of separate metadata, 5316 bytes of string pool
Choosing a maxzoom of -z0 for features about 3626844 feet (1105462 meters) apart
Choosing a maxzoom of -z4 for resolution of about 31866 feet (9712 meters) within features
  99.9%  4/4/5  


In [0]:
# Upload to mapbox
# Get processed boundaries data
#!rm -r $MA_BUCKET/boundaries/processed/geojson
#!gsutil -m cp -r gs://$MA_BUCKET/boundaries/processed/geojson $MA_BUCKET/boundaries/processed
# Rename GEE exports  
#!mv ma-data/boundaries/processed/geojson/selected_EEZ_land_v2_201410_simplified.json ma-data/boundaries/processed/geojson/selected_EEZ_land_v2_201410.json
#!mv ma-data/boundaries/processed/geojson/selected_WDPA_polygons_ee_export.json ma-data/boundaries/processed/geojson/selected_WDPA_polygons.json  
    
import glob
path = '{}/boundaries/processed/geojson'.format(os.environ.get('MA_BUCKET'))
#files = [f for f in glob.glob(path + "**/*.json", recursive=True)]
#for f in files:
#  upload_to_mapbox(f, os.path.splitext(os.path.basename(f))[0])
files = [f for f in glob.glob(path + "**/*.mbtiles", recursive=True)]
for f in files:
  print(f)
  upload_to_mapbox(f, os.path.splitext(os.path.basename(f))[0])  

mangrove_atlas/boundaries/processed/geojson/selected_WDPA_polygons.mbtiles
{'message': 'Not Found'}
mangrove_atlas/boundaries/processed/geojson/selected_EEZ_land_v2_201410.mbtiles
{'message': 'Not Found'}


In [0]:
%%bash
# Clean up
rm -r $MA_BUCKET/boundaries

In [0]:
ls mangrove_atlas

#Prepare mangrove land-cover data

The original data files contain many interesect, island and sliver errors and can be simplified. This section parses the .ZIP files, cleans and simplifies them using mapshaper. Adds a property year, combines all periods into a single feature collection, converts to different formats and adds to GEE and MapBox.

+ Data files (.ZIP packaged shape files) should be added to Google Cloud Storage (GCS) bucket
+ Files are downloaded, processed using mapshaper, uploaded to GCS bucket, and added as GEE asset.



In [0]:
%%bash
# Get land-cover data
gsutil -m cp -r gs://$MA_BUCKET/land-cover $MA_BUCKET

In [0]:
!rm -r $MA_BUCKET/land-cover/shapefiles
!rm -r $MA_BUCKET/land-cover/processed

rm: cannot remove 'mangrove_atlas/land-cover/shapefiles': No such file or directory


In [0]:
!ls $MA_BUCKET/land-cover

OneDrive_1_5-22-2019.zip  OneDrive_4_5-22-2019.zip  OneDrive_5_5-27-2019.zip
OneDrive_2_5-22-2019.zip  OneDrive_4_5-27-2019.zip
OneDrive_3_5-22-2019.zip  OneDrive_5_5-22-2019.zip


## Simplify and convert shapefiles

In [0]:
%%bash
# Process zipped shapefiles
# Clean and simplfy
# Convert to geojson
# change mapshaper settings below, see https://github.com/mbloch/mapshaper/wiki/Command-Reference
# NOTE this can take along time!

# Set working directories
df=$MA_BUCKET/land-cover
sf=$MA_BUCKET/land-cover/shapefiles
pf=$MA_BUCKET/land-cover/processed

# Create directories (if needed)
mkdir $sf
mkdir $pf
mkdir -p ${pf}/geojson

# Extract original ZIP to $sf
arr=(${df}/*.zip)
for ((i=0; i<${#arr[@]}; i++)); do
  z="${arr[$i]}"
  echo "Extracting" $z
  unzip -o $z -d $sf
done

# Clean and simplify original SHP
arr=(${sf}/*.shp)
for ((i=0; i<${#arr[@]}; i++)); do
  s="${arr[$i]}"
  echo "Simplifying" ${s}
  # FIXME RUN WITH LOW SIMPLIFICATION
  mapshaper-xl \
  -i snap files="${s}" \
  -simplify weighted interval=$SIMP_WEIGHT_INT \
  -clean min-gap-area=$MIN_AREA \
  -filter-islands min-area=$MIN_AREA \
  -filter-slivers min-area=$MIN_AREA \
  -clean min-gap-area=$MIN_AREA \
  -o ${pf} format=shapefile precision=0.000001 force
done  

# Export processed SHP as geojson to $pf/geojson
arr=(${pf}/*.shp)
for ((i=0; i<${#arr[@]}; i++)); do
  sp="${arr[$i]}"
  echo "Converting" ${sp}
  mapshaper-xl \
  -i files="${sp}" \
  -o ${pf}/geojson format=geojson precision=0.000001 force
done


In [0]:
# Zip processed shape files
%%bash
fl=($MA_BUCKET/land-cover/processed/*.shp)
echo ${fl}
for ((i=0; i<${#fl[@]}; i++)); do
  z="${fl[$i]}"
  bn="${z%%.*}"
  zip $bn.zip $bn.*
  
done

ma-data/land-cover/processed/*.shp
	zip warning: name not matched: ma-data/land-cover/processed/*.*

zip error: Nothing to do! (ma-data/land-cover/processed/*.zip)


In [0]:
%%bash
# Upload to GCS bucket
pf=$MA_BUCKET/land-cover/processed
gsutil -m cp ${pf}/*.zip gs://${pf}/
gsutil -m cp -r ${pf}/geojson gs://${pf}/

## Combine simplified geojson files

In [0]:
%%bash
# Download from GCS bucket
pf=$MA_BUCKET/land-cover/processed
mkdir -p $pf
gsutil -m cp -r gs://${pf} $MA_BUCKET/land-cover


In [0]:
# Add year property to each layer
import glob
pf=os.environ.get('MA_BUCKET') + '/land-cover/processed/geojson/*.json'
files = (glob.glob(pf))
#print(files)
for f in files:
  print(f)
  with open(f) as of:
    data = json.load(of)
    year = int(f.split('_')[2])
    print(year)
    #inspect
    #print(data['features'][0])

    updated_features = [{**d, 'properties': {**d['properties'], 'year': year}} for d in data['features']]

    data['features'] = updated_features
    #print(data['features'][0])

    with open(f, 'w') as wf:
      json.dump(data, wf)
  

mangrove_atlas/land-cover/processed/geojson/GMW_2010_v2.json
2010
mangrove_atlas/land-cover/processed/geojson/GMW_2008_v2.json
2008
mangrove_atlas/land-cover/processed/geojson/GMW_1996_v2.json
1996
mangrove_atlas/land-cover/processed/geojson/GMW_2007_v2.json
2007
mangrove_atlas/land-cover/processed/geojson/GMW_2009_v2.json
2009
mangrove_atlas/land-cover/processed/geojson/GMW_2016_v2.json
2016
mangrove_atlas/land-cover/processed/geojson/GMW_2015_v2.json
2015


In [0]:
# Join geojsons using Tippecanoe
%%bash
# Make mapbox tiles
tippecanoe \
-zg \
-o $MA_BUCKET/land-cover/processed/geojson/gmw_1996_2016.mbtiles \
--drop-densest-as-needed \
--extend-zooms-if-still-dropping \
--force \
--read-parallel \
-l gmw_1996_2016 \
$MA_BUCKET/land-cover/processed/geojson/*.json



1981041 features, 362774430 bytes of geometry, 22640990 bytes of separate metadata, 4941420 bytes of string pool
Choosing a maxzoom of -z8 for features about 1043 feet (318 meters) apart
Choosing a maxzoom of -z10 for resolution of about 477 feet (145 meters) within features
tile 7/58/59 size is 507987 with detail 12, >500000    
Going to try keeping the sparsest 88.58% of the features to make it fit
tile 8/211/125 size is 632415 with detail 12, >500000    
Going to try keeping the sparsest 71.16% of the features to make it fit
tile 8/70/109 size is 556272 with detail 12, >500000    
Going to try keeping the sparsest 80.90% of the features to make it fit
tile 8/132/124 size is 520052 with detail 12, >500000    
Going to try keeping the sparsest 86.53% of the features to make it fit
  99.9%  10/808/513  


In [0]:
%%bash
# Upload to GCS bucket
pf=$MA_BUCKET/land-cover/processed/geojson/gmw_1996_2016.mbtiles
gsutil -m cp -r ${pf} gs://${pf}


Copying file://mangrove_atlas/land-cover/processed/geojson/gmw_1996_2016.mbtiles [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

|
Operation completed over 1 objects/161.5 MiB.                                    


In [0]:
# Add to Mapbox

# Get latest mbtiles
!gsutil -m cp -r gs://$MA_BUCKET/land-cover/processed/geojson/gmw_1996_2016.mbtiles $MA_BUCKET/land-cover/processed/geojson
    
import glob 
import os
path = os.environ.get('MA_BUCKET') + '/land-cover/processed/geojson'
files = [f for f in glob.glob(path + "**/*.mbtiles", recursive=True)]
for f in files:
  upload_to_mapbox(f, os.path.splitext(os.path.basename(f))[0])

Copying gs://mangrove_atlas/land-cover/processed/geojson/gmw_1996_2016.mbtiles...
/ [1/1 files][161.5 MiB/161.5 MiB] 100% Done                                    
Operation completed over 1 objects/161.5 MiB.                                    
{'message': 'Not Found'}


In [0]:
ls mangrove_atlas/land-cover/processed

geojson/               gmw_1996_2016.prj  GMW_1996_v2.zip  GMW_2010_v2.zip
gmw_1996_2016.dbf      gmw_1996_2016.shp  GMW_2007_v2.zip  GMW_2015_v2.zip
gmw_1996_2016.geojson  gmw_1996_2016.shx  GMW_2008_v2.zip  GMW_2016_v2.zip
gmw_1996_2016.mbtiles  gmw_1996_2016.zip  GMW_2009_v2.zip


In [0]:
%%bash 
# Combine geojsons and convert to shapefile
mapshaper-xl \
    -i $MA_BUCKET/land-cover/processed/geojson/*.json snap combine-files \
    -rename-layers gmw1996,gmw2007,gmw2008,gmw2009,gmw2010,gmw2015,gmw2016 \
    -merge-layers target=gmw1996,gmw2007,gmw2008,gmw2009,gmw2010,gmw2015,gmw2016 name=gmw_1996_2016 \
    -o $MA_BUCKET/land-cover/processed/gmw_1996_2016.shp format=shapefile target=gmw_1996_2016 precision=0.000001 force


Allocating 8000 megabytes of heap memory
[i] Snapped 0 points
[o] Wrote mangrove_atlas/land-cover/processed/gmw_1996_2016.shp
[o] Wrote mangrove_atlas/land-cover/processed/gmw_1996_2016.shx
[o] Wrote mangrove_atlas/land-cover/processed/gmw_1996_2016.dbf
[o] Wrote mangrove_atlas/land-cover/processed/gmw_1996_2016.prj


In [0]:
# Zip processed shape files
%%bash
pf=$MA_BUCKET/land-cover/processed
fl=(${pf}/*.shp)
for ((i=0; i<${#fl[@]}; i++)); do
  z="${fl[$i]}"
  bn="${z%%.*}"
  zip $bn.zip $bn.*
  
done

  adding: mangrove_atlas/land-cover/processed/gmw_1996_2016.dbf (deflated 79%)
  adding: mangrove_atlas/land-cover/processed/gmw_1996_2016.prj (deflated 10%)
  adding: mangrove_atlas/land-cover/processed/gmw_1996_2016.shp (deflated 68%)
  adding: mangrove_atlas/land-cover/processed/gmw_1996_2016.shx (deflated 48%)


In [0]:
ls mangrove_atlas/land-cover/processed/

geojson/           gmw_1996_2016.shx  GMW_2008_v2.zip  GMW_2016_v2.zip
gmw_1996_2016.dbf  gmw_1996_2016.zip  GMW_2009_v2.zip
gmw_1996_2016.prj  GMW_1996_v2.zip    GMW_2010_v2.zip
gmw_1996_2016.shp  GMW_2007_v2.zip    GMW_2015_v2.zip


In [0]:
%%bash
# Upload to GCS
pf=$MA_BUCKET/land-cover/processed/gmw_1996_2016.zip
gsutil -m cp -r ${pf} gs://${pf}

Copying file://mangrove_atlas/land-cover/processed/gmw_1996_2016.zip [Content-Type=application/zip]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\
Operation completed over 1 objects/335.9 MiB.                                    


In [0]:
# Add as GEE asset
# single year layers take upto 10 min
# combined layer takes about 17m
import os
# TODO ADD METADATA!
file_path="gs://" + os.environ.get('MA_BUCKET') + "/land-cover/processed/*.zip"
asset_path="projects/global-mangrove-watch/land-cover"
append_txt="_simplified"
strProv="Global Mangrove Watch"
strVer=2.0
strDOI="=http://dx.doi.org/10.7910/DVN/HKGBGS"
strCite="Bunting P., Rosenqvist A., Lucas R., Rebelo L-M., Hilarides L., Thomas N., Hardy A., Itoh T., Shimada M. and Finlayson C.M. (2018). The Global Mangrove Watch – a New 2010 Global Baseline of Mangrove Extent. Remote Sens. 2018, 10, 1669; doi:10.3390/rs10101669"

gee_upload_zips_to_tables(file_path, asset_path, time_start="1996-01-01T00:00:00", force=True, append_txt=append_txt, properties={"(string)provider": strProv, "version": strVer, "(string)doi": strDOI, "(string)citation": strCite})

Found 9 files
Processing GMW_1996_v2
earthengine --no-use_cloud_api upload table --asset_id=projects/global-mangrove-watch/land-cover/GMW_1996_v2_simplified --force --time_start=1996-01-01T00:00:00 --property='(string)provider=Global Mangrove Watch' --property='version=2.0' --property='(string)doi==http://dx.doi.org/10.7910/DVN/HKGBGS' --property='(string)citation=Bunting P., Rosenqvist A., Lucas R., Rebelo L-M., Hilarides L., Thomas N., Hardy A., Itoh T., Shimada M. and Finlayson C.M. (2018). The Global Mangrove Watch – a New 2010 Global Baseline of Mangrove Extent. Remote Sens. 2018, 10, 1669; doi:10.3390/rs10101669'  gs://mangrove_atlas/land-cover/processed/GMW_1996_v2.zip
Task created
Processing GMW_2007_v2
earthengine --no-use_cloud_api upload table --asset_id=projects/global-mangrove-watch/land-cover/GMW_2007_v2_simplified --force --time_start=1996-01-01T00:00:00 --property='(string)provider=Global Mangrove Watch' --property='version=2.0' --property='(string)doi==http://dx.doi.or

In [0]:
# Clean up
!rm -r $MA_BUCKET/land-cover

---
#Prepare mangrove land-cover change data

Once combined land-cover asset is uploaded to GEE, the JS code `users/vizzuality/mangrove-atlas/land-cover-mangrove-gain-loss.js` creates change layers between consecutive time periods and exports them to `https://storage.cloud.google.com/mangrove_atlas/land-cover/processed/geojson/`

+ All change layers are combined into a single feature collection with fields `start_year`, `end_year`, and `change_typ`.

In [0]:
%%bash
# Download from GCS bucket
pf=$MA_BUCKET/land-cover/processed/geojson/gain
mkdir -p $pf
gsutil -m cp -r gs://${pf} $MA_BUCKET/land-cover/processed/geojson

pf=$MA_BUCKET/land-cover/processed/geojson/loss
mkdir -p $pf
gsutil -m cp -r gs://${pf} $MA_BUCKET/land-cover/processed/geojson

ls $MA_BUCKET/land-cover/processed/geojson

gain
loss


Copying gs://mangrove_atlas/land-cover/processed/geojson/gain/gl_2007_1996_gain.geojson...
Copying gs://mangrove_atlas/land-cover/processed/geojson/gain/gl_2015_2010_gain.geojson...
Copying gs://mangrove_atlas/land-cover/processed/geojson/gain/gl_2010_2009_gain.geojson...
Copying gs://mangrove_atlas/land-cover/processed/geojson/gain/gl_2008_2007_gain.geojson...
Copying gs://mangrove_atlas/land-cover/processed/geojson/gain/gl_2016_2015_gain.geojson...
Copying gs://mangrove_atlas/land-cover/processed/geojson/gain/gl_2009_2008_gain.geojson...
Copying gs://mangrove_atlas/land-cover/processed/geojson/gain/gl_2015_2016_gain_.geojson...
Copying gs://mangrove_atlas/land-cover/processed/geojson/loss/gl_2010_2015_loss.geojson...
Copying gs://mangrove_atlas/land-cover/processed/geojson/loss/gl_2009_2010_loss.geojson...
Copying gs://mangrove_atlas/land-cover/processed/geojson/loss/gl_2007_2008_loss.geojson...
Copying gs://mangrove_atlas/land-cover/processed/geojson/loss/gl_1996_2007_loss.geojson..

In [0]:
%%bash
mv $MA_BUCKET/land-cover/processed/geojson/gain/*.geojson $MA_BUCKET/land-cover/processed/geojson
mv $MA_BUCKET/land-cover/processed/geojson/loss/*.geojson $MA_BUCKET/land-cover/processed/geojson

In [0]:
ls $MA_BUCKET/land-cover/processed/geojson/

gain/                      gl_2010_2009_gain.geojson
gl_1996_2007_loss.geojson  gl_2010_2015_loss.geojson
gl_2007_1996_gain.geojson  gl_2015_2010_gain.geojson
gl_2007_2008_loss.geojson  gl_2015_2016_gain_.geojson
gl_2008_2007_gain.geojson  gl_2015_2016_loss.geojson
gl_2008_2009_loss.geojson  gl_2016_2015_gain.geojson
gl_2009_2008_gain.geojson  loss/
gl_2009_2010_loss.geojson


In [0]:
# Fix order of start and end time
import json
import glob
import os
p = f"{os.environ.get('MA_BUCKET')}/land-cover/processed/geojson/*.geojson"
fl = glob.glob(p)
for f in fl:
  data = json.load(open(f))
  for ft in data['features']: 
    tmp_sy = ft.get('properties').get('start_year')
    tmp_ey = ft.get('properties').get('end_year')
    tmp_t = ft.get('properties').get('change_type')
    tmp_ti = ft.get('properties').get('time_interval')
    #print(tmp_sy)
    #print(tmp_ey)
    #print(tmp_ti)
    #print(tmp_t)
    if tmp_t == 'loss':
      tmp_ti = f"{tmp_sy}--{tmp_ey}"
    if tmp_t == 'gain':
      ge = tmp_sy
      gs = tmp_ey
      tmp_sy = gs
      tmp_ey = ge
      tmp_ti = f"{tmp_sy}--{tmp_ey}"  
    #print(tmp_sy)
    #print(tmp_ey)
    #print(tmp_ti)
    #print(tmp_t)
    ft['properties']['start_year'] = tmp_sy
    ft['properties']['end_year'] = tmp_ey
    ft['properties']['time_interval'] = tmp_ti
  with open(f, 'w') as outfile:
    json.dump(data, outfile)
   

In [0]:
%%bash 
# Combine geojsons and convert to shapefile
mapshaper-xl \
    -i $MA_BUCKET/land-cover/processed/geojson/*.geojson snap combine-files \
    -merge-layers name=gmw_change_1996_2016 force \
    -o $MA_BUCKET/land-cover/processed/gmw_change_1996_2016.shp format=shapefile target=gmw_change_1996_2016 precision=0.000001 rfc7946 force

Allocating 8 GB of heap memory
[i] Snapped 1707770 points
[o] Changed field name from "change_type" to "change_typ"
[o] Changed field name from "time_interval" to "time_inter"
[o] Wrote mangrove_atlas/land-cover/processed/gmw_change_1996_2016.shp
[o] Wrote mangrove_atlas/land-cover/processed/gmw_change_1996_2016.shx
[o] Wrote mangrove_atlas/land-cover/processed/gmw_change_1996_2016.dbf
[o] Wrote mangrove_atlas/land-cover/processed/gmw_change_1996_2016.prj


In [0]:
ls $MA_BUCKET/land-cover/processed

geojson/                  gmw_change_1996_2016.prj  gmw_change_1996_2016.shx
gmw_change_1996_2016.dbf  gmw_change_1996_2016.shp


In [0]:
 # Zip processed shape files
%%bash
pf=$MA_BUCKET/land-cover/processed
fl=(${pf}/*.shp)
for ((i=0; i<${#fl[@]}; i++)); do
  z="${fl[$i]}"
  bn="${z%%.*}"
  zip $bn.zip $bn.*
  
done

updating: mangrove_atlas/land-cover/processed/gmw_change_1996_2016.dbf (deflated 89%)
updating: mangrove_atlas/land-cover/processed/gmw_change_1996_2016.prj (deflated 10%)
updating: mangrove_atlas/land-cover/processed/gmw_change_1996_2016.shp (deflated 53%)
updating: mangrove_atlas/land-cover/processed/gmw_change_1996_2016.shx (deflated 19%)


In [0]:
%%bash
# Upload to GCS
pf=$MA_BUCKET/land-cover/processed/gmw_change_1996_2016.zip
gsutil -m cp -r ${pf} gs://${pf}

Copying file://mangrove_atlas/land-cover/processed/gmw_change_1996_2016.zip [Content-Type=application/zip]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/
Operation completed over 1 objects/211.0 MiB.                                    


In [0]:
# Add as GEE asset
# single year layers take upto 10 min
# combined layer takes about 17m
import os
# TODO ADD METADATA!
file_path=f"gs://{os.environ.get('MA_BUCKET')}/land-cover/processed/gmw_change_1996_2016.zip"
asset_path="projects/global-mangrove-watch/land-cover"
append_txt="_simplified"
strProv="Global Mangrove Watch"
strVer=2.0
strDOI="=http://dx.doi.org/10.7910/DVN/HKGBGS"
strCite="Bunting P., Rosenqvist A., Lucas R., Rebelo L-M., Hilarides L., Thomas N., Hardy A., Itoh T., Shimada M. and Finlayson C.M. (2018). The Global Mangrove Watch – a New 2010 Global Baseline of Mangrove Extent. Remote Sens. 2018, 10, 1669; doi:10.3390/rs10101669"

gee_upload_zips_to_tables(file_path, asset_path, time_start="1996-01-01T00:00:00", force=True, append_txt=append_txt, properties={"(string)provider": strProv, "version": strVer, "(string)doi": strDOI, "(string)citation": strCite})

Found 2 files
Processing gmw_change_1996_2016
earthengine --no-use_cloud_api upload table --asset_id=projects/global-mangrove-watch/land-cover/gmw_change_1996_2016_simplified --force --time_start=1996-01-01T00:00:00 --property='(string)provider=Global Mangrove Watch' --property='version=2.0' --property='(string)doi==http://dx.doi.org/10.7910/DVN/HKGBGS' --property='(string)citation=Bunting P., Rosenqvist A., Lucas R., Rebelo L-M., Hilarides L., Thomas N., Hardy A., Itoh T., Shimada M. and Finlayson C.M. (2018). The Global Mangrove Watch – a New 2010 Global Baseline of Mangrove Extent. Remote Sens. 2018, 10, 1669; doi:10.3390/rs10101669'  gs://mangrove_atlas/land-cover/processed/gmw_change_1996_2016.zip
Task created
Processing 
earthengine --no-use_cloud_api upload table --asset_id=projects/global-mangrove-watch/land-cover/_simplified --force --time_start=1996-01-01T00:00:00 --property='(string)provider=Global Mangrove Watch' --property='version=2.0' --property='(string)doi==http://dx.d

In [0]:
ls $MA_BUCKET/land-cover/processed/geojson

gain/                       gl_2009_2010_gain_.geojson
gl_1996_2007_gain_.geojson  gl_2010_2009_loss_.geojson
gl_2007_1996_loss_.geojson  gl_2010_2015_gain_.geojson
gl_2007_2008_gain_.geojson  gl_2015_2010_loss_.geojson
gl_2008_2007_loss_.geojson  gl_2015_2016_gain_.geojson
gl_2008_2009_gain_.geojson  gl_2016_2015_loss_.geojson
gl_2009_2008_loss_.geojson  loss/


In [0]:
# Join geojsons using Tippecanoe
%%bash
# Make mapbox tiles
tippecanoe \
-zg \
-o $MA_BUCKET/land-cover/processed/geojson/gmw_change_1996_2016.mbtiles \
--drop-densest-as-needed \
--extend-zooms-if-still-dropping \
--force \
--read-parallel \
-l gmw_change_1996_2016 \
$MA_BUCKET/land-cover/processed/geojson/*.geojson

1156 features, 160934384 bytes of geometry, 17440 bytes of separate metadata, 696 bytes of string pool
Choosing a maxzoom of -z5 for features about 8482 feet (2586 meters) apart
Choosing a maxzoom of -z10 for resolution of about 367 feet (111 meters) within features
  99.9%  10/260/461  


In [0]:
%%bash
# Upload to GCS bucket
pf=$MA_BUCKET/land-cover/processed/geojson/gmw_change_1996_2016.mbtiles
gsutil -m cp -r ${pf} gs://${pf}


Copying file://mangrove_atlas/land-cover/processed/geojson/gmw_change_1996_2016.mbtiles [Content-Type=application/octet-stream]...
|
Operation completed over 1 objects/95.6 MiB.                                     


In [0]:
# Add to Mapbox

# Get latest mbtiles
!gsutil -m cp -r gs://$MA_BUCKET/land-cover/processed/geojson/gmw_change_1996_2016.mbtiles $MA_BUCKET/land-cover/processed/geojson
    
import glob 
import os
path = os.environ.get('MA_BUCKET') + '/land-cover/processed/geojson'
files = [f for f in glob.glob(path + "**/*.mbtiles", recursive=True)]
for f in files:
  upload_to_mapbox(f, os.path.splitext(os.path.basename(f))[0])

Copying gs://mangrove_atlas/land-cover/processed/geojson/gmw_change_1996_2016.mbtiles...
- [1/1 files][ 95.6 MiB/ 95.6 MiB] 100% Done                                    
Operation completed over 1 objects/95.6 MiB.                                     
{'message': 'Not Found'}


In [0]:
# Clean up
!rm -r $MA_BUCKET/land-cover

# Prepare mangrove properties data

+ Year 200 data is Simard et al. 2019 download data from https://daac.ornl.gov/cgi-bin/dsviewer.pl?ds_id=1665 and transfer .ZIP to GCS
    + Aboveground mangrove biomass (AGB, Mg Ha-1)
    + maximum canopy height (height of the tallest tree) (Hmax, m)
    + basal-area weighted height (individual tree heights weighted in proportion to their basal area) (HBA, m)
 
+ New data for more years (1996, 2007, 2008, 2009, 2010, 2015, 2016) supplied by Pete Bunting 
+ Prepare metadata and add as GEE asset


In [0]:
%%bash
# Get mangrove-properties data from FTP and add to GCS
#curl -u eogroup:aberdata 'ftp://144.124.80.198//PeteBunting/gmw_hgt_agb/TIF/gmw_agb_simard_etal_tif.tar.gz' -o gmw_agb_simard_etal_tif.tar.gz
#curl -u eogroup:aberdata 'ftp://144.124.80.198//PeteBunting/gmw_hgt_agb/TIF/gmw_hgt_simard_etal_tif.tar.gz' -o gmw_hgt_simard_etal_tif.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1412M  100 1412M    0     0  2780k      0  0:08:40  0:08:40 --:--:-- 2227k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4130M  100 4130M    0     0  9087k      0  0:07:45  0:07:45 --:--:-- 14.5M


In [0]:
ls -lhS mangrove_atlas/mangrove-properties/images

total 0


In [0]:
%%bash
# Transfer to GCS
#gsutil -m cp -r gmw_agb_simard_etal_tif.tar.gz gs://$MA_BUCKET/mangrove-properties/
#gsutil -m cp -r gmw_hgt_simard_etal_tif.tar.gz gs://$MA_BUCKET/mangrove-properties/

In [0]:
%%bash
#rm gmw_agb_simard_etal_tif.tar.gz
#rm gmw_hgt_simard_etal_tif.tar.gz

## Above ground biomass

In [0]:
%%bash
# Get mangrove-properties data
mkdir -p $MA_BUCKET/mangrove-properties/images
gsutil -m cp -r gs://$MA_BUCKET/mangrove-properties/gmw_agb_simard_etal_tif.tar.gz $MA_BUCKET/mangrove-properties

Copying gs://mangrove_atlas/mangrove-properties/gmw_agb_simard_etal_tif.tar.gz...
\ [1/1 files][  1.4 GiB/  1.4 GiB] 100% Done  44.5 MiB/s ETA 00:00:00           
Operation completed over 1 objects/1.4 GiB.                                      


In [0]:
%%bash
tar xvzf $MA_BUCKET/mangrove-properties/gmw_agb_simard_etal_tif.tar.gz -C $MA_BUCKET/mangrove-properties/images

In [0]:
%%bash
# Push back to GCS (so we can ingest in GEE!!)
y=1996
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_agb_simard_etal/${y}/*_agb_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_aboveground_biomass_${y}_v1-0/
y=2007
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_agb_simard_etal/${y}/*_agb_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_aboveground_biomass_${y}_v1-0/
y=2008
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_agb_simard_etal/${y}/*_agb_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_aboveground_biomass_${y}_v1-0/
y=2009
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_agb_simard_etal/${y}/*_agb_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_aboveground_biomass_${y}_v1-0/
y=2010
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_agb_simard_etal/${y}/*_agb_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_aboveground_biomass_${y}_v1-0/
y=2015
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_agb_simard_etal/${y}/*_agb_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_aboveground_biomass_${y}_v1-0/
y=2016
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_agb_simard_etal/${y}/*_agb_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_aboveground_biomass_${y}_v1-0/ 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
%%bash
# Clean up
rm -r $MA_BUCKET/mangrove-properties/gmw_agb_simard_etal_tif.tar.gz
rm -r $MA_BUCKET/mangrove-properties/images 

In [0]:
import os
# Upload to GEE imageCollecion asset
# @todo check all images have been uploaded to ImageCollection
# @todo check if there is a 200 tasks limit


# Define asset path
assetP="projects/global-mangrove-watch/mangrove-properties"

# Define common metadata for the layers
# for units use definitions in https://www.unidata.ucar.edu/software/udunits/udunits-current/udunits/
properties = {'version':"'1.0'",'provider':"'GMW'",'doi':"''",'citation':"''",'resolution':"'30m'"}
nodata_value=0
pyramiding_policy="mean"

# Define specific metadata and generate tasks
# AGB
#year_list = [2007,2008,2009,2010,2015,2016]
#for year in year_list:
#  file_path="gs://{}/mangrove-properties/processed/mangrove_aboveground_biomass_1996-2016/*.tif".format(os.environ.get('MA_BUCKET'), year)
  #print(file_path)
#  asset_path="projects/global-mangrove-watch/mangrove-properties/mangrove_aboveground_biomass_{}_v1-0_collection".format(year)
  #print(asset_path)
#  properties['units']="'milligram per hectare'"
#  bands="agb_biomass"
  # time_start should be ISO8061
#  time_start="{}-01-01T12:00:00".format(year)
  #print(properties)
#  gee_upload_images_to_collection(file_path, asset_path, pyramiding_policy, bands, time_start, nodata_value, force=True, properties=properties)

# Hmax
year_list = [1996,2007,2008,2009,2010,2015,2016]
for year in year_list:
  file_path="gs://{}/mangrove-properties/processed/mangrove_max_canopy_height_1996-2016/mangrove_max_canopy_height_{}_v1-0.tif".format(os.environ.get('MA_BUCKET'), year)
  #print(file_path)
  asset_path="projects/global-mangrove-watch/mangrove-properties/mangrove_max_canopy_height_1996-2016/mangrove_max_canopy_height_{}_v1-0".format(year)
  #print(asset_path)
  properties['units']="'metres'"
  bands="max_canopy_height"
  # time_start should be ISO8061
  time_start="{}-01-01T12:00:00".format(year)
  #print(properties)
  gee_upload_single_image(file_path, asset_path, pyramiding_policy, bands, time_start, nodata_value, force=True, properties=properties)


# HBA
#year_list = [1996,2007,2008,2009,2010,2015,2016]
#for year in year_list:
#  file_path="gs://{}/mangrove-properties/processed/mangrove_basal-area_weighted_height_{}_v1-0/*.tif".format(os.environ.get('MA_BUCKET'), year)
  #print(file_path)
#  asset_path="projects/global-mangrove-watch/mangrove-properties/mangrove_basal-area_weighted_height_biomass_{}_v1-0_collection".format(year)
  #print(asset_path)
#  properties['units']="'metres'"
#  bands="ba_weighted_height"
  # time_start should be ISO8061
#  time_start="{}-01-01T12:00:00".format(year)
  #print(properties)
#  gee_upload_images_to_collection(file_path, asset_path, pyramiding_policy, bands, time_start, nodata_value, force=True, properties=properties)



Found 2 files
Processing projects/global-mangrove-watch/mangrove-properties/mangrove_max_canopy_height_1996-2016/mangrove_max_canopy_height_1996_v1-0
earthengine --no-use_cloud_api upload image --asset_id=projects/global-mangrove-watch/mangrove-properties/mangrove_max_canopy_height_1996-2016/mangrove_max_canopy_height_1996_v1-0 --force --pyramiding_policy=mean --time_start=1996-01-01T12:00:00 --nodata_value=0 --bands=max_canopy_height --property=version='1.0' --property=provider='GMW' --property=doi='' --property=citation='' --property=resolution='30m' --property=units='metres' gs://mangrove_atlas/mangrove-properties/processed/mangrove_max_canopy_height_1996-2016/mangrove_max_canopy_height_1996_v1-0.tif 
Task created
Finished upload
Found 2 files
Processing projects/global-mangrove-watch/mangrove-properties/mangrove_max_canopy_height_1996-2016/mangrove_max_canopy_height_2007_v1-0
earthengine --no-use_cloud_api upload image --asset_id=projects/global-mangrove-watch/mangrove-properties/m

In [0]:
!earthengine --no-use_cloud_api upload image --asset_id=projects/global-mangrove-watch/mangrove-properties/mangrove_max_canopy_height_1996-2016/mangrove_max_canopy_height_1996_v1-0 --force --pyramiding_policy=mean --time_start=1996-01-01T12:00:00 --nodata_value=0 --bands=max_canopy_height --property=version='1.0' --property=provider='GMW' --property=doi='' --property=citation='' --property=resolution='30m' --property=units='metres' 'gs://mangrove_atlas/mangrove-properties/processed/mangrove_max_canopy_height_1996-2016/mangrove_max_canopy_height_1996_v1-0.tif'



Started upload task with ID: OBP3RQ2WMC7QCNS3MJQCJY5Z


In [0]:
import os
# Upload to GEE imageCollecion asset
# @todo check all images have been uploaded to ImageCollection
# @todo check if there is a 200 tasks limit


# Define asset path
assetP="projects/global-mangrove-watch/mangrove-properties"

# Define common metadata for the layers
# for units use definitions in https://www.unidata.ucar.edu/software/udunits/udunits-current/udunits/
properties = {'version':"'1.0'",'provider':"'GMW'",'doi':"''",'citation':"''",'resolution':"'30m'"}
nodata_value=0
pyramiding_policy="mean"

# Define specific metadata and generate tasks

# Hmax
year_list = [1996,2007,2008,2009,2010,2015,2016]
for year in year_list:
  file_path="gs://{}/mangrove-properties/processed/mangrove_maximum_canopy_height_{}_v1-0/*.tif".format(os.environ.get('MA_BUCKET'), year)
  #print(file_path)
  asset_path="projects/global-mangrove-watch/mangrove-properties/mangrove_maximum_canopy_height_biomass_{}_v1-0_collection".format(year)
  #print(asset_path)
  properties['units']="'metres'"
  bands="max_canopy_height"
  # time_start should be ISO8061
  time_start="{}-01-01T12:00:00".format(year)
  #print(properties)
  gee_upload_images_to_collection(file_path, asset_path, pyramiding_policy, bands, time_start, nodata_value, force=True, properties=properties)




## Canopy height

In [0]:
%%bash
# Get mangrove-properties data
mkdir -p $MA_BUCKET/mangrove-properties/images
gsutil -m cp -r gs://$MA_BUCKET/mangrove-properties/gmw_hgt_simard_etal_tif.tar.gz $MA_BUCKET/mangrove-properties
tar xvzf $MA_BUCKET/mangrove-properties/gmw_hgt_simard_etal_tif.tar.gz -C $MA_BUCKET/mangrove-properties/images
rm -r $MA_BUCKET/mangrove-properties/gmw_hgt_simard_etal_tif.tar.gz

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Copying gs://mangrove_atlas/mangrove-properties/gmw_hgt_simard_etal_tif.tar.gz...
| [1/1 files][  4.0 GiB/  4.0 GiB] 100% Done  28.6 MiB/s ETA 00:00:00           
Operation completed over 1 objects/4.0 GiB.                                      


In [0]:
ls -lhS $MA_BUCKET/mangrove-properties/images/gmw_hgt_simard_etal/hba/1996

In [0]:
%%bash
# Push back to GCS (so we can ingest in GEE!!)
# hba
y=1996
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_hgt_simard_etal/hba/${y}/*_hgt_hba_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_basal-area_weighted_height_${y}_v1-0/
y=2007
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_hgt_simard_etal/hba/${y}/*_hgt_hba_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_basal-area_weighted_height_${y}_v1-0/
y=2008
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_hgt_simard_etal/hba/${y}/*_hgt_hba_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_basal-area_weighted_height_${y}_v1-0/
y=2009
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_hgt_simard_etal/hba/${y}/*_hgt_hba_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_basal-area_weighted_height_${y}_v1-0/
y=2010
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_hgt_simard_etal/hba/${y}/*_hgt_hba_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_basal-area_weighted_height_${y}_v1-0/
y=2015
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_hgt_simard_etal/hba/${y}/*_hgt_hba_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_basal-area_weighted_height_${y}_v1-0/
y=2016
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_hgt_simard_etal/hba/${y}/*_hgt_hba_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_basal-area_weighted_height_${y}_v1-0/

# hmax
y=1996
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_hgt_simard_etal/hmax/${y}/*_hgt_hmax_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_max_canopy_height_${y}_v1-0/
  y=2007
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_hgt_simard_etal/hmax/${y}/*_hgt_hmax_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_max_canopy_height_${y}_v1-0/
  y=2008
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_hgt_simard_etal/hmax/${y}/*_hgt_hmax_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_max_canopy_height_${y}_v1-0/
  y=2009
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_hgt_simard_etal/hmax/${y}/*_hgt_hmax_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_max_canopy_height_${y}_v1-0/
  y=2010
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_hgt_simard_etal/hmax/${y}/*_hgt_hmax_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_max_canopy_height_${y}_v1-0/
  y=2015
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_hgt_simard_etal/hmax/${y}/*_hgt_hmax_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_max_canopy_height_${y}_v1-0/
  y=2016
gsutil -m cp $MA_BUCKET/mangrove-properties/images/gmw_hgt_simard_etal/hmax/${y}/*_hgt_hmax_gmw_${y}.tif \
gs://$MA_BUCKET/mangrove-properties/processed/mangrove_max_canopy_height_${y}_v1-0/

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
%%bash
# Clean up
rm -r $MA_BUCKET/mangrove-properties/gmw_hgt_simard_etal_tif.tar.gz
rm -r $MA_BUCKET/mangrove-properties/images

## Year 2000 data

In [0]:
%%bash
# Get mangrove-properties data
#mkdir -p $MA_BUCKET/mangrove-properties/images
#gsutil -m cp -r gs://$MA_BUCKET/mangrove-properties/CMS_Global_Map_Mangrove_Canopy_1665.zip $MA_BUCKET/mangrove-properties

Copying gs://ma-data/mangrove-properties/CMS_Global_Map_Mangrove_Canopy_1665.zip...
| [1/1 files][  1.2 GiB/  1.2 GiB] 100% Done  84.4 MiB/s ETA 00:00:00           
Operation completed over 1 objects/1.2 GiB.                                      


In [0]:
!ls ma-data/mangrove-properties/images

In [0]:
# Unzip images
%time
#import shutil
#import os
#p1 = os.environ.get('MA_BUCKET') + '/mangrove-properties/CMS_Global_Map_Mangrove_Canopy_1665.zip' 
#p2 = os.environ.get('MA_BUCKET') + 'mangrove-properties/images' 
#shutil.unpack_archive(p1, p2)
#unzip -o ma-data/mangrove-properties/CMS_Global_Map_Mangrove_Canopy_1665.zip -d ma-data/mangrove-properties/images

In [0]:
%%bash
# Merge into single rasters
#mkdir -p ma-data/mangrove-properties/processed

# AGB
#gdalwarp \
#ma-data/mangrove-properties/images/CMS_Global_Map_Mangrove_Canopy_1665/data/Mangrove_agb_*.tif \
#ma-data/mangrove-properties/processed/mangrove_aboveground_biomass_2000_v1-2.tif \
#-srcnodata 0 \
#-dstnodata 0

Creating output file that is 1296001P x 252001L.


ERROR 3: Free disk space available is 14858256384 bytes, whereas 1306374192004 are at least necessary. You can disable this check by defining the CHECK_DISK_FREE_SPACE configuration option to FALSE.


In [0]:
# Push back to GCS (so we can ingest in GEE!!)
#!gsutil -m cp ma-data/mangrove-properties/images/CMS_Global_Map_Mangrove_Canopy_1665/data/Mangrove_agb_*.tif gs://ma-data/mangrove-properties/processed/mangrove_aboveground_biomass_2000_v1-2
#!gsutil -m cp ma-data/mangrove-properties/images/CMS_Global_Map_Mangrove_Canopy_1665/data/Mangrove_hmax95_*.tif gs://ma-data/mangrove-properties/processed/mangrove_max_canopy_height_2000_v1-2
#!gsutil -m cp ma-data/mangrove-properties/images/CMS_Global_Map_Mangrove_Canopy_1665/data/Mangrove_hba95_*.tif gs://ma-data/mangrove-properties/processed/mangrove_basal-area_weighted_height_2000_v1-2  

In [0]:
# Check info for an image
!gdalinfo ma-data/mangrove-properties/images/CMS_Global_Map_Mangrove_Canopy_1665/data/Mangrove_agb_Brazil.tif

In [0]:
def get_gs_files(path, extension, format_manifest=False):
  
  cmd = "".join(["gsutil ls ", path,  "/*", extension])
  print(cmd)
  out=os.popen(cmd).read().split('\n')
  if format_manifest:
    out = [{'primaryPath': source} for source in out]
  return out  

get_gs_files("gs://ma-data/mangrove-properties/processed/mangrove_aboveground_biomass_2000_v1-2", ".tif", True)

In [0]:
# Prepare upload manifest for each layer
import os

def get_gs_files(path, extension, format_manifest=False):
  
  cmd = "".join(["gsutil ls ", path,  "/*", extension])
  #print(cmd)
  out=os.popen(cmd).read().split('\n')
  if format_manifest:
    out = [{'primaryPath': source} for source in out]
  return out  

# Get GCS asset file list
agb_path="gs://ma-data/mangrove-properties/processed/mangrove_aboveground_biomass_2000_v1-2"
hmax_path="gs://ma-data/mangrove-properties/processed/mangrove_max_canopy_height_2000_v1-2"
hba_path="gs://ma-data/mangrove-properties/processed/mangrove_basal-area_weighted_height_2000_v1-2"

files_agb = get_gs_files(agb_path, ".tif", True)
files_hmax = get_gs_files(hmax_path, ".tif", True)
files_hba =  get_gs_files(hba_path, ".tif", True)

# Define base asset path
assetP="projects/global-mangrove-watch/mangrove-properties"

# Define common metadata for the layers
# for units use definitions in https://www.unidata.ucar.edu/software/udunits/udunits-current/udunits/
strVer="1.2"
strProv="NASA-ORNL-DAAC"
strDOI="https://doi.org/10.3334/ORNLDAAC/1665"
strCite="<p>Simard et al. (2019) “Mangrove canopy height globally related to precipitation, temperature and cyclone frequency.” Nature Geoscience 12:40-45. <a href=\"https://daac.ornl.gov/cgi-bin/dsviewer.pl?ds_id=1665\">Download</a></p>"
strRes="30m"
# time_start should be ISO8061
strTS="2000-01-01T12:00:00"
nodata_value=0
pyramiding_policy="mean"


In [0]:
# Create upload manifest for biomass
# AGB
assetID = "mangrove_aboveground_biomass_2000_v1-2"
data_agb = {
  "id": assetP + "/" + assetID,
  "pyramidingPolicy" : "mean",  
  "bands": [
    {
      "id": "agb_biomass"
    }
  ],
  "missingData": {
    "value": "0"
  },
  "properties": {
    "provider": strProv,
    "version" : strVer,
    "citation" : strCite,
    "doi" : strDOI,  
    "resolution" : strRes,
    "system:time_start": strTS,
    "gcs_path": agb_path,
    "units" : "milligram per hectare",
    "product_type": "raster",
    "number_of_bands": "1",
    "epsg_code": "",
    "resampling_kernel": "",
    "number_of_rows": "",
    "number_of_columns": ""},
  "tilesets": [
    {
      "sources": [
        files_agb
      ]
    }
  ]
}

import json
f = assetID + '.json'
print(f)
with open(f, 'w') as outfile:
    json.dump(data_agb, outfile)
    
cmd = "gsutil cp " + f + " gs://ma-data/ee-upload-manifests/"
print(cmd)
os.system(cmd)

mangrove_aboveground_biomass_2000_v1-2.json
gsutil cp mangrove_aboveground_biomass_2000_v1-2.json gs://ma-data/ee-upload-manifests/


0

In [0]:
# Create upload manifest for canopy height
assetID = "mangrove_max_canopy_height_2000_v1-2"
data_hmax = {
  "id": os.path.join(assetP, assetID),
  "pyramidingPolicy" : "mean",  
  "bands": [
    {
      "id": "max_canopy_height"
    }
  ],
  "missingData": {
    "value": "0"
  },
  "properties": {
    "provider": strProv,
    "version" : strVer,
    "citation" : strCite,
    "doi" : strDOI,  
    "resolution" : strRes,
    "system:time_start": strTS,
    "gcs_path": hmax_path,
    "units" : "meter",
    "product_type": "raster",
    "number_of_bands": "1",
    "epsg_code": "",
    "resampling_kernel": "",
    "number_of_rows": "",
    "number_of_columns": ""},
  "tilesets": [
    {
      "sources": [
        files_hmax
      ]
    }
  ]
}

import json
f = assetID + '.json'
print(f)
with open(f, 'w') as outfile:
    json.dump(data_agb, outfile)
    
cmd = "gsutil cp " + f + " gs://ma-data/ee-upload-manifests/"
print(cmd)
os.system(cmd)

mangrove_max_canopy_height_2000_v1-2.json
gsutil cp mangrove_max_canopy_height_2000_v1-2.json gs://ma-data/ee-upload-manifests/


0

In [0]:
# Create upload manifest for ba height
assetID = "mangrove_basal-area_weighted_height_2000_v1-2"
data_hmax = {
  "id": os.path.join(assetP, assetID),
  "pyramidingPolicy" : "mean",  
  "bands": [
    {
      "id": "ba_weighted_height"
    }
  ],
  "missingData": {
    "value": "0"
  },
  "properties": {
    "provider": strProv,
    "version" : strVer,
    "citation" : strCite,
    "doi" : strDOI,  
    "resolution" : strRes,
    "system:time_start": strTS,
    "gcs_path": hba_path,
    "units" : "meter",
    "product_type": "raster",
    "number_of_bands": "1",
    "epsg_code": "",
    "resampling_kernel": "",
    "number_of_rows": "",
    "number_of_columns": ""},
  "tilesets": [
    {
      "sources": [
        files_hba
      ]
    }
  ]
}

import json
f = assetID + '.json'
print(f)
with open(f, 'w') as outfile:
    json.dump(data_agb, outfile)
    
cmd = "gsutil cp " + f + " gs://ma-data/ee-upload-manifests/"
print(cmd)
os.system(cmd)

mangrove_basal-area_weighted_height_2000_v1-2.json
gsutil cp mangrove_basal-area_weighted_height_2000_v1-2.json gs://ma-data/ee-upload-manifests/


0

In [0]:

!earthengine upload image --force --manifest mangrove_aboveground_biomass_2000_v1-2.json

Malformed JSON provided.


In [0]:
cat mangrove_aboveground_biomass_2000_v1-2.json

{"id": "projects/global-mangrove-watch/mangrove-properties/mangrove_aboveground_biomass_2000_v1-2", "pyramidingPolicy": "mean", "bands": [{"id": "agb_biomass"}], "missingData": {"value": "0"}, "properties": {"provider": "NASA-ORNL-DAAC", "version": "1.2", "citation": "<p>Simard et al. (2019) \u201cMangrove canopy height globally related to precipitation, temperature and cyclone frequency.\u201d Nature Geoscience 12:40-45. <a href=\"https://daac.ornl.gov/cgi-bin/dsviewer.pl?ds_id=1665\">Download</a></p>", "doi": "https://doi.org/10.3334/ORNLDAAC/1665", "resolution": "30m", "system:time_start": "2000-01-01T12:00:00", "gcs_path": "gs://ma-data/mangrove-properties/processed/mangrove_aboveground_biomass_2000_v1-2", "units": "milligram per hectare", "product_type": "raster", "number_of_bands": "1", "epsg_code": "", "resampling_kernel": "", "number_of_rows": "", "number_of_columns": ""}, "tilesets": [{"sources": [[{"primaryPath": "gs://ma-data/mangrove-properties/processed/mangrove_abovegroun

In [0]:
%%bash
# Upload to GEE imageCollecion asset
# @todo check all images have been uploaded to ImageCollection
# @todo check if there is a 200 tasks limit


# Define asset path
assetP="projects/global-mangrove-watch/mangrove-properties"

# Define common metadata for the layers
# for units use definitions in https://www.unidata.ucar.edu/software/udunits/udunits-current/udunits/
strVer="version=1.2"
strProv="(string)provider=NASA-ORNL-DAAC"
strDOI="(string)doi=https://doi.org/10.3334/ORNLDAAC/1665"
strCite="(string)citation=<p>Simard et al. (2019) “Mangrove canopy height globally related to precipitation, temperature and cyclone frequency.” Nature Geoscience 12:40-45. <a href=\"https://daac.ornl.gov/cgi-bin/dsviewer.pl?ds_id=1665\">Download</a></p>"
strRes="(string)resolution=30m"
# time_start should be ISO8061
strTS="2000-01-01T12:00:00"
nodata_value=0
pyramiding_policy="mean"

# Define specific metadata and generate tasks
# AGB
GCS_PATH="gs://ma-data/mangrove-properties/processed/mangrove_aboveground_biomass_2000_v1-2/*.tif"
assetId="mangrove_aboveground_biomass_2000_v1-2_collection"
strUnits="(string)units=milligram per hectare"
bandId="agb_biomass"
#gee_upload_images_to_collection

# Hmax
GCS_PATH="gs://ma-data/mangrove-properties/processed/mangrove_max_canopy_height_2000_v1-2/*.tif"
assetId="mangrove_max_canopy_height_2000_v1-2_collection"
strUnits="(string)units=meter"
bandId="max_canopy_height"
#gee_upload_images_to_collection

# HBA
GCS_PATH="gs://ma-data/mangrove-properties/processed/mangrove_basal-area_weighted_height_2000_v1-2/*.tif"
assetId="mangrove_basal-area_weighted_height_2000_v1-2_collection"
strUnits="(string)units=meter"
bandId="ba_weighted_height"
gee_upload_images_to_collection


In [0]:
# Export to single images in GEE
import ee
ee.Initialize()
agb = ee.ImageCollection("projects/global-mangrove-watch/mangrove-properties/mangrove_aboveground_biomass_2000_v1-2").mosaic()
nr = agb.projection().nominalScale()
geom = agb.geometry().bounds() 
task = ee.batch.Export.image.toAsset(agb, "convert agb2000", "projects/global-mangrove-watch/mangrove-properties/mangrove_aboveground_biomass_2000_v1-2_image", "mean", None, geom, nr, None, None, 1e12)
task.start()
hmax = ee.ImageCollection("projects/global-mangrove-watch/mangrove-properties/mangrove_max_canopy_height_2000_v1-2").mosaic()
nr = hmax.projection().nominalScale()
geom = hmax.geometry().bounds()
ee.batch.Export.image.toAsset(hmax, "convert hmax2000", "projects/global-mangrove-watch/mangrove-properties/mangrove_max_canopy_height_2000_v1-2_image", "mean", None, geom, nr, None, None, 1e12)
task.start()
hba = ee.ImageCollection("projects/global-mangrove-watch/mangrove-properties/mangrove_basal-area_weighted_height_2000_v1-2").mosaic()
nr = hba.projection().nominalScale()
geom = hba.geometry().bounds()
ee.batch.Export.image.toAsset(hba, "convert hba2000", "projects/global-mangrove-watch/mangrove-properties/mangrove_basal-area_weighted_height_2000_v1-2_image", "mean", None, geom, nr, None, None, 1e12)
task.start()

TypeError: ignored

## Add geotiffs to MapBox

In [0]:
%%bash
# Get data
mkdir -p $MA_BUCKET/mangrove-properties/images
gsutil -m cp -r gs://$MA_BUCKET/mangrove-properties/processed/mangrove_aboveground_biomass_2000_v1-2 $MA_BUCKET/mangrove-properties/images


In [0]:
# Clean up
rm -r ma-data/mangrove-properties

In [0]:
ls $MA_BUCKET/mangrove-properties/images/mangrove_aboveground_biomass_2000_v1-2

Mangrove_agb_AndamanAndNicobar.tif
Mangrove_agb_Angola.tif
Mangrove_agb_Anguilla.tif
Mangrove_agb_AntiguaAndBarbuda.tif
Mangrove_agb_Aruba.tif
Mangrove_agb_Australia.tif
Mangrove_agb_Bahamas.tif
Mangrove_agb_Bahrain.tif
Mangrove_agb_Bangladesh.tif
Mangrove_agb_Barbados.tif
Mangrove_agb_Belize.tif
Mangrove_agb_Benin.tif
Mangrove_agb_Brazil.tif
Mangrove_agb_BritishVirginIslands.tif
Mangrove_agb_Brunei.tif
Mangrove_agb_Cambodia.tif
Mangrove_agb_Cameroon.tif
Mangrove_agb_CarribeanCaymanIslands.tif
Mangrove_agb_China.tif
Mangrove_agb_Colombia.tif
Mangrove_agb_Comoros.tif
Mangrove_agb_CostaRica.tif
Mangrove_agb_CoteDivoire.tif
Mangrove_agb_Cuba.tif
Mangrove_agb_DemocraticRepublicOfCongo.tif
Mangrove_agb_Djibouti.tif
Mangrove_agb_DominicanRepublic.tif
Mangrove_agb_EcuadorWithGalapagos.tif
Mangrove_agb_Egypt.tif
Mangrove_agb_ElSalvador.tif
Mangrove_agb_EquatorialGuinea.tif
Mangrove_agb_Eritrea.tif
Mangrove_agb_EuropaIsland.tif
Mangrove_agb_Fiji2.tif
Mangrove_agb_Fiji.tif
Mangrove_agb_FrenchGuy

In [0]:
# Add to Mapbox
    
import glob 
import os
path = os.environ.get('MA_BUCKET') + '/mangrove-properties/images/mangrove_aboveground_biomass_2000_v1-2/'
files = [f for f in glob.glob(path + "**/*.tif", recursive=True)]
for f in files[0:2]:
  print(f)
  print(os.path.splitext(os.path.basename(f))[0])
  upload_to_mapbox(f, os.path.splitext(os.path.basename(f))[0])

mangrove_atlas/mangrove-properties/images/mangrove_aboveground_biomass_2000_v1-2/Mangrove_agb_Macau.tif
Mangrove_agb_Macau
{'message': 'Not Found'}
mangrove_atlas/mangrove-properties/images/mangrove_aboveground_biomass_2000_v1-2/Mangrove_agb_Grenada.tif
Mangrove_agb_Grenada
{'message': 'Not Found'}


# Process elevation rasters

+ upload to GCS bucket
+ add as GEE asset (parsing information from rasters on disk)

In [0]:
%%bash
# Get elevation data
gsutil -m cp -r gs://ma-data/elevation ma-data


Copying gs://ma-data/elevation/aoi_saloum_dem11.tif...


In [0]:
%%bash

# Get resolution info from given raster
function get_pixelsize() {
    if [ -z "$1" ]; then
        echo "Missing arguments. Syntax:"
        echo "  get_pixelsize <input_raster>"
        return
    fi
    EXTENT=( $(gdalinfo "$1" |\
        grep "Pixel Size" |\
        sed "s/Pixel Size =//g; s/,/ /g" |\
        tr "\n" " " |\
        tr -d "[(,])-") )
    echo -n "$EXTENT"
}

# Get no data info from given raster
function get_navalue() {
    if [ -z "$1" ]; then
        echo "Missing arguments. Syntax:"
        echo "  get_pixelsize <input_raster>"
        return
    fi
    EXTENT=( $(gdalinfo "$1" |\
        grep "NoData Value" |\
        sed "s/NoData Value=//g; s/,/ /g" |\
        tr "\n" " " |\
        tr -d "[(,])") )
    echo -n "$EXTENT"
}


# TODO get more metadata for each image!
# Upload multple images to GEE asset as collection!
# Get file list
bn_asset=ma-data/elevation
eims=`gsutil ls gs://$bn_asset/*.tif`

# Define asset path
assetP="projects/global-mangrove-watch/elevation"

# Define metadata for the layers
# for units use definitions in https://www.unidata.ucar.edu/software/udunits/udunits-current/udunits/
#strVer="(string)version="
strProv="(string)provider='Wetlands International'"
#strDOI="(string)doi="
#strCite="(string)citation="
# time_start should be ISO8061
#strTS=""
#nodata_value=0
pyramiding_policy="mean"

# Orthoimage DSM
assetIdElev="dsm_orthoimages"
strUnitsElev="(string)units=centimeter"
earthengine create collection $assetP/$assetIdElev

for file in $eims; do
  filename=`basename $file`
  asset_id="${filename%.*}"
  res=$(get_pixelsize ${file#gs://})
  nodata_value=$(get_navalue ${file#gs://})
  strRes="resolution=$res"
  earthengine upload image --asset_id="$assetP/$assetIdElev/$asset_id" \
  --force \
  --pyramiding_policy=$pyramiding_policy \
  --nodata_value=$nodata_value \
  --property="${strProv}" \
  --property="${strRes}" \
  --property="${strUnitsElev}" \
  $file

done  

Asset projects/global-mangrove-watch/elevation/dsm_orthoimages already exists.
Started upload task with ID: ZAM2ARGETKVXGFPWCJGXCYZM


In [0]:
# Clean up
rm-r ma-data/elevation

# Process UAV orthoimage rasters

+ upload to GCS bucket
+ add as GEE image collection asset (parsing information from rasters on disk)

In [0]:
%%bash
# Get orthoimagery data
gsutil -m cp -r gs://ma-data/orthoimagery ma-data


Copying gs://ma-data/orthoimagery/aoi_rufiji_Nyamisati.tif...
Copying gs://ma-data/orthoimagery/aoi_rufiji_Rice_field.tif...
Copying gs://ma-data/orthoimagery/aoi_saloum_ortho.tif...
Copying gs://ma-data/orthoimagery/aoi_rufiji_Mixed_Veg_Rufiji_delta.tif...


In [0]:
!gdalinfo ma-data/orthoimagery/aoi_rufiji_Nyamisati.tif

Driver: GTiff/GeoTIFF
Files: ma-data/orthoimagery/aoi_rufiji_Nyamisati.tif
Size is 20019, 17473
Coordinate System is:
PROJCS["WGS 84 / UTM zone 37S",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            TOWGS84[0,0,0,0,0,0,0],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0],
        UNIT["degree",0.0174532925199433],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",39],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",10000000],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AUTHORITY["EPSG","32737"]]
Origin = (530506.043157355161384,9138589.096017528325319)
Pixel Size = (0.011132366273810,-0.011132366273778)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  COMPRESSION=LZW
  INTERLEAVE=PIXE

In [0]:
%%bash

# Get resolution info from given raster
function get_pixelsize() {
    if [ -z "$1" ]; then
        echo "Missing arguments. Syntax:"
        echo "  get_pixelsize <input_raster>"
        return
    fi
    EXTENT=( $(gdalinfo "$1" |\
        grep "Pixel Size" |\
        sed "s/Pixel Size =//g; s/,/ /g" |\
        tr "\n" " " |\
        tr -d "[(,])-") )
    echo -n "$EXTENT"
}

# TODO get more metadata for each image! USE MANIFEST JSON!!
# Upload multple images to GEE asset as collection!
# Get file list
bn_asset=ma-data/orthoimagery
eims=`gsutil ls gs://$bn_asset/*.tif`

# Define asset path
assetP="projects/global-mangrove-watch/orthoimagery"

# Define metadata for the layers
# for units use definitions in https://www.unidata.ucar.edu/software/udunits/udunits-current/udunits/
#strVer="(string)version="
strProv="(string)provider='Wetlands International'"
#strDOI="(string)doi="
#strCite="(string)citation="
# time_start should be ISO8061
#strTS=""
pyramiding_policy="mean"

# Orthoimages
assetIdCol="uav_orthoimages"
strUnits="(string)units=RGBcolor"
earthengine create collection $assetP/$assetIdCol

for file in $eims; do
  filename=`basename $file`
  asset_id="${filename%.*}"
  res=$(get_pixelsize ${file#gs://})
  echo $res
  strRes="resolution=$res"
  earthengine upload image --asset_id="$assetP/$assetIdCol/$asset_id" \
  --force \
  --last_band_alpha \
  --pyramiding_policy=$pyramiding_policy \
  --property="${strProv}" \
  --property="${strRes}" \
  --property="${strUnits}" \
  $file

done  

Asset projects/global-mangrove-watch/orthoimagery/uav_orthoimages already exists.
0.015169615226314
Started upload task with ID: URPC5MZH4W7V33Z2BOD3CFDU
0.011132366273810
Started upload task with ID: 4LW7E7NYLONCJ7DF64BS7J5J
0.012607628398661
Started upload task with ID: TW4DEG4IQQ7TDGCANGTQZ2ZJ
0.018485947526993
Started upload task with ID: Q73W3UROQJCYZ3K2GVGQKPHM


In [0]:
# Clean up
!rm -r ma-data/orthoimagery

# Coastline vector line strings

OSM coastlines - regularily updated, detailed in some regions, poor in others!

+ https://osmdata.openstreetmap.de/download/coastlines-split-4326.zip


Global Self-consistent, Hierarchical, High-resolution Geography Database (GSHHG) - stable, not regularily updated

+ https://www.ngdc.noaa.gov/mgg/shorelines/data/gshhg/latest/gshhg-shp-2.3.7.zip

In [0]:
# Get OSM coastlines and transfer to GCS
!mkdir ma-data/physical-environment 
!curl https://osmdata.openstreetmap.de/download/coastlines-split-4326.zip -o ma-data/physical-environment/osm_coastlines-split-4326.zip

mkdir: cannot create directory ‘ma-data/physical-environment’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  533M  100  533M    0     0  12.7M      0  0:00:41  0:00:41 --:--:-- 13.7M


In [0]:
# Get GSHHG
!mkdir ma-data/physical-environment
!curl https://www.ngdc.noaa.gov/mgg/shorelines/data/gshhg/latest/gshhg-shp-2.3.7.zip -o ma-data/physical-environment/gshhg-shp-2.3.7.zip

mkdir: cannot create directory ‘ma-data/physical-environment’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  142M  100  142M    0     0  13.7M      0  0:00:10  0:00:10 --:--:-- 14.0M


In [0]:
# Upload to GCS
!gsutil -m cp -r ma-data/physical-environment gs://ma-data

In [0]:
%%bash
# Unzip and select shapefiles
# Set working directories
df=ma-data/physical-environment
sf=ma-data/physical-environment/shapefiles
pf=ma-data/physical-environment/processed

# Create directories (if needed)
mkdir $df
mkdir $sf
mkdir $pf
mkdir -p ${pf}/geojson

# Pull from GCS
gsutil -m cp -r gs://ma-data/physical-environment ma-data

# Extract original ZIP to $sf
arr=(${df}/*.zip)
for ((i=0; i<${#arr[@]}; i++)); do
  z="${arr[$i]}"
  echo "Extracting" $z
  unzip -o $z -d $sf
done


Extracting ma-data/physical-environment/gshhg-shp-2.3.7.zip
Archive:  ma-data/physical-environment/gshhg-shp-2.3.7.zip
  inflating: ma-data/physical-environment/shapefiles/README.TXT  
  inflating: ma-data/physical-environment/shapefiles/SHAPEFILES.TXT  
  inflating: ma-data/physical-environment/shapefiles/LICENSE.TXT  
  inflating: ma-data/physical-environment/shapefiles/COPYING.LESSERv3  
  inflating: ma-data/physical-environment/shapefiles/GSHHS_shp/c/GSHHS_c_L1.dbf  
  inflating: ma-data/physical-environment/shapefiles/GSHHS_shp/c/GSHHS_c_L1.prj  
  inflating: ma-data/physical-environment/shapefiles/GSHHS_shp/c/GSHHS_c_L1.shp  
  inflating: ma-data/physical-environment/shapefiles/GSHHS_shp/c/GSHHS_c_L1.shx  
  inflating: ma-data/physical-environment/shapefiles/GSHHS_shp/c/GSHHS_c_L2.dbf  
  inflating: ma-data/physical-environment/shapefiles/GSHHS_shp/c/GSHHS_c_L2.prj  
  inflating: ma-data/physical-environment/shapefiles/GSHHS_shp/c/GSHHS_c_L2.shp  
  inflating: ma-data/physical-en

mkdir: cannot create directory ‘ma-data/physical-environment’: File exists
mkdir: cannot create directory ‘ma-data/physical-environment/processed’: File exists
Copying gs://ma-data/physical-environment/gshhg-shp-2.3.7.zip...
Copying gs://ma-data/physical-environment/osm_coastlines-split-4326.zip...


In [0]:
ls ma-data/physical-environment/shapefiles

coastlines-split-4326/  GSHHS_shp/   README.TXT      WDBII_shp/
COPYING.LESSERv3        LICENSE.TXT  SHAPEFILES.TXT


In [0]:
%%bash
# Process zipped shapefiles
# Clean and simplfy
# Convert to geojson
# change mapshaper settings below, see https://github.com/mbloch/mapshaper/wiki/Command-Reference
# NOTE this can take along time!

# Set working directories
df=ma-data/physical-environment
sf=ma-data/physical-environment/shapefiles
pf=ma-data/physical-environment/processed

# Clean and simplify original SHP
osm=coastlines-split-4326
arr=(${sf}/${osm}/*.shp)
for ((i=0; i<${#arr[@]}; i++)); do
  s="${arr[$i]}"
  echo "Simplifying" ${s}
  mapshaper-xl \
  -i snap files="${s}" \
  -simplify weighted interval=100 \
  -clean min-gap-area=1000 \
  -filter-islands min-area=1000 \
  -filter-slivers min-area=1000 \
  -clean min-gap-area=1000 \
  -o ${pf} format=shapefile precision=0.000001 force
done  

# Export processed SHP as geojson to $pf/geojson
arr=(${pf}/*.shp)
for ((i=0; i<${#arr[@]}; i++)); do
  sp="${arr[$i]}"
  echo "Converting" ${sp}
  mapshaper-xl \
  -i files="${sp}" \
  -o ${pf}/geojson format=geojson precision=0.000001 force
done


Extracting ma-data/physical-environment/gshhg-shp-2.3.7.zip
Archive:  ma-data/physical-environment/gshhg-shp-2.3.7.zip
  inflating: ma-data/physical-environment/shapefiles/README.TXT  
  inflating: ma-data/physical-environment/shapefiles/SHAPEFILES.TXT  
  inflating: ma-data/physical-environment/shapefiles/LICENSE.TXT  
  inflating: ma-data/physical-environment/shapefiles/COPYING.LESSERv3  
  inflating: ma-data/physical-environment/shapefiles/GSHHS_shp/c/GSHHS_c_L1.dbf  
  inflating: ma-data/physical-environment/shapefiles/GSHHS_shp/c/GSHHS_c_L1.prj  
  inflating: ma-data/physical-environment/shapefiles/GSHHS_shp/c/GSHHS_c_L1.shp  
  inflating: ma-data/physical-environment/shapefiles/GSHHS_shp/c/GSHHS_c_L1.shx  
  inflating: ma-data/physical-environment/shapefiles/GSHHS_shp/c/GSHHS_c_L2.dbf  
  inflating: ma-data/physical-environment/shapefiles/GSHHS_shp/c/GSHHS_c_L2.prj  
  inflating: ma-data/physical-environment/shapefiles/GSHHS_shp/c/GSHHS_c_L2.shp  
  inflating: ma-data/physical-en

Allocating 8000 megabytes of heap memory
Error: [i] No files matched (ma-data/physical-environment/shapefiles/*.shp)
Run mapshaper -h to view help
Allocating 8000 megabytes of heap memory
Error: [i] No files matched (ma-data/physical-environment/processed/*.shp)
Run mapshaper -h to view help


In [0]:
# Add as GEE assets
file_path="gs://ma-data/physical-environment/gshhg-shp-2.3.7.zip"
asset_path="projects/global-mangrove-watch/physical-environment"
append_txt=""
strProv="NOAA"
strVer="2.3.7"
strDOI="https://www.ngdc.noaa.gov/mgg/shorelines/"
strCite="Wessel, P., and W. H. F. Smith (1996), A global, self-consistent, hierarchical, high-resolution shoreline database, J. Geophys. Res., 101(B4), 8741–8743, doi:10.1029/96JB00104"
#gee_upload_zips_to_tables(file_path, asset_path, time_start="2017-06-15T00:00:00", force=True, append_txt=append_txt, properties={"(string)provider": strProv, "version": strVer, "(string)doi": strDOI, "(string)citation": strCite})

file_path="gs://ma-data/physical-environment/osm_coastlines-split-4326.zip"
asset_path="projects/global-mangrove-watch/physical-environment"
append_txt=""
strProv="OSM"
strVer="2019-05-28T03:16"
strDOI="https://osmdata.openstreetmap.de/data/coastlines.html"
strCite="OpenStreetMap data and OpenStreetMap contributors"
gee_upload_zips_to_tables(file_path, asset_path, time_start="2019-05-28T03:16:00", force=True, append_txt=append_txt, properties={"(string)provider": strProv, "version": strVer, "(string)doi": strDOI, "(string)citation": strCite})

Found 2 files
Processing gshhg-shp-2.3.7
earthengine upload table --asset_id=projects/global-mangrove-watch/physical-environment/gshhg-shp-2.3.7 --force --time_start=2017-06-15T00:00:00 --property='(string)provider=NOAA' --property='version=2.3.7' --property='(string)doi=https://www.ngdc.noaa.gov/mgg/shorelines/' --property='(string)citation=Wessel, P., and W. H. F. Smith (1996), A global, self-consistent, hierarchical, high-resolution shoreline database, J. Geophys. Res., 101(B4), 8741–8743, doi:10.1029/96JB00104'  gs://ma-data/physical-environment/gshhg-shp-2.3.7.zip
Task created
Processing 
earthengine upload table --asset_id=projects/global-mangrove-watch/physical-environment/ --force --time_start=2017-06-15T00:00:00 --property='(string)provider=NOAA' --property='version=2.3.7' --property='(string)doi=https://www.ngdc.noaa.gov/mgg/shorelines/' --property='(string)citation=Wessel, P., and W. H. F. Smith (1996), A global, self-consistent, hierarchical, high-resolution shoreline datab

# Process GEE analysis output

+ at the moment this is a GEE JS code 
    + https://code.earthengine.google.com/be95d4d02efb430e186e5406c3d3b46a
    + https://code.earthengine.google.com/?accept_repo=users/vizzuality/mangrove-atlas

In [0]:
# Get gee analysis output
import os
import subprocess

def get_gee_export(bdir, adir):
  try:
    p = 'gs://{0}/{1}/{2}'.format(os.environ.get('MA_BUCKET'), bdir, adir)
    d = '{0}/{1}'.format(os.environ.get('MA_BUCKET'), bdir)
    # make directory if it does not exist
    if not os.path.exists(d):
      os.makedirs(d)
    s = subprocess.run(['gsutil', '-m', 'cp', '-r', p, d], check=True)
    print('COMPLETED: {0} transfered TO {1}'.format(p,d))
  except subprocess.CalledProcessError as err:
    print('ERROR:', err)


bdir = 'ee-export-tables'

# land-cover
get_gee_export(bdir, 'land-cover')
get_gee_export(bdir, 'mangrove-properties')
get_gee_export(bdir, 'environmental-pressures')
!gsutil cp gs://mangrove_atlas/ee-export-tables/locations_aoi-country-wdpa_20190917.geojson mangrove_atlas/ee-export-tables/


COMPLETED: gs://mangrove_atlas/ee-export-tables/land-cover transfered TO mangrove_atlas/ee-export-tables
COMPLETED: gs://mangrove_atlas/ee-export-tables/mangrove-properties transfered TO mangrove_atlas/ee-export-tables
COMPLETED: gs://mangrove_atlas/ee-export-tables/environmental-pressures transfered TO mangrove_atlas/ee-export-tables
Copying gs://mangrove_atlas/ee-export-tables/locations_aoi-country-wdpa_20190917.geojson...
- [1 files][  9.1 MiB/  9.1 MiB]                                                
Operation completed over 1 objects/9.1 MiB.                                      


In [0]:
%%bash 
# Combine geojsons and convert to geojson
mapshaper-xl \
    -i $MA_BUCKET//ee-export-tables/land-cover/*.geojson combine-files \
    -merge-layers \
    -o $MA_BUCKET//ee-export-tables/land-cover_data.geojson format=geojson force

mapshaper-xl \
    -i $MA_BUCKET//ee-export-tables/mangrove-properties/*.geojson combine-files \
    -merge-layers \
    -o $MA_BUCKET//ee-export-tables/mangrove-properties.geojson format=geojson force

mapshaper-xl \
    -i $MA_BUCKET//ee-export-tables/environmental-pressures/*.geojson combine-files \
    -merge-layers \
    -o $MA_BUCKET//ee-export-tables/environmental-pressures.geojson format=geojson force



Allocating 8000 megabytes of heap memory
[o] Wrote mangrove_atlas/ee-export-tables/land-cover_data.geojson
Allocating 8000 megabytes of heap memory
[o] Wrote mangrove_atlas/ee-export-tables/mangrove-properties.geojson
Allocating 8000 megabytes of heap memory
[merge-layers] Error: Inconsistent data types in "con_hotspot_summary_km2" field: object, string
Run mapshaper -h to view help


In [0]:
%%bash
mapshaper-xl -i $MA_BUCKET/ee-export-tables/locations_aoi-country-wdpa_20190917.geojson \
    -join $MA_BUCKET//ee-export-tables/land-cover_data.geojson keys=id,id string-fields=id  \
    -o $MA_BUCKET/ee-export-tables/mangrove_atlas_data.geojson force


Allocating 8000 megabytes of heap memory
[join] Error: External table is missing a field named: id
Run mapshaper -h to view help


In [0]:
%%bash
gsutil cp $MA_BUCKET/ee-export-tables/land-cover_data.geojson gs://$MA_BUCKET/ee-export-tables/land-cover_data.json
gsutil cp $MA_BUCKET/ee-export-tables/mangrove-properties.geojson gs://$MA_BUCKET/ee-export-tables/mangrove-properties_data.json
gsutil cp $MA_BUCKET/ee-export-tables/environmental-pressures.geojson gs://$MA_BUCKET/ee-export-tables/environmental-pressures_data.json

Copying file://mangrove_atlas/ee-export-tables/land-cover_data.geojson [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/174.8 KiB.                                    
Copying file://mangrove_atlas/ee-export-tables/mangrove-properties.geojson [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/898.2 KiB.                                    
CommandException: No URLs matched: mangrove_atlas/ee-export-tables/environmental-pressures.geojson


In [0]:
ls -lh mangrove_atlas/ee-export-tables/

total 11M
drwxr-xr-x 2 root root  24K Sep 24 17:42 environmental-pressures/
drwxr-xr-x 2 root root  28K Sep 24 17:42 land-cover/
-rw-r--r-- 1 root root 175K Sep 24 17:46 land-cover_data.geojson
-rw-r--r-- 1 root root 9.1M Sep 24 17:42 locations_aoi-country-wdpa_20190917.geojson
drwxr-xr-x 2 root root  20K Sep 24 17:42 mangrove-properties/
-rw-r--r-- 1 root root 899K Sep 24 17:46 mangrove-properties.geojson


In [0]:
# Get gee analysis output
# FIXME MAKE GEE OUTPUT FILE AUTOMATIC NAMED
import os
import subprocess
bdir = 'ee-export-tables'
b_file_path = 'locations_aoi-country-wdpa_20190924'
p = 'gs://' + os.environ.get('MA_BUCKET') + '/' + bdir + '/' + b_file_path + '.geojson'
subprocess.run(['gsutil', 'cp', p, '.'])


CompletedProcess(args=['gsutil', 'cp', 'gs://mangrove_atlas/ee-export-tables/locations_aoi-country-wdpa_20190924.geojson', '.'], returncode=0)

In [0]:
ls

locations_aoi-country-wdpa_20190924.geojson  mangrove_atlas/


In [0]:
# Convert LOCATIONS table from geojson to a flat CSV structure
import json
import pandas as pd

data=json.load(open(b_file_path + '.geojson'))['features']

df_list = []
for f in data:
  geometry = f.get('geometry')
  #print(geometry)
  ids = f.get('id')
  props = f.get('properties')
  #print(props.keys())
  bounds = props['bounds']
  del props['bounds']
  #print(props)
  out = pd.DataFrame.from_records([props])
  out['id'] = ids
  out['geometry'] = json.dumps(geometry)
  out['bounds'] = json.dumps(bounds)
  df_list.append(out)
  
df = pd.concat(df_list)
df = df.rename(columns={"type": "location_type", "length_coast_m": "coast_length_m", "id": "location_id"})
#df = df[["name", "location_type", "iso", "bounds", "geometry", "area_m2", "perimeter_m", "coast_length_m", "location_id"]]

df
df.to_csv(b_file_path + '.csv', index=False, na_rep="NA", sep=";")

In [0]:
# add to GCS
# FIXME MAKE GEE OUTPUT FILE AUTOMATIC NAMED
import os
import subprocess
p = 'gs://' + os.environ.get('MA_BUCKET') + '/' + bdir + '/' + b_file_path + '.csv'
subprocess.run(['gsutil', 'cp', b_file_path + '.csv', p])

CompletedProcess(args=['gsutil', 'cp', 'locations_aoi-country-wdpa_20190924.csv', 'gs://mangrove_atlas/ee-export-tables/locations_aoi-country-wdpa_20190924.csv'], returncode=1)

In [0]:
df[['location_id']].size == pd.unique(df['location_id']).size

True

In [0]:
%%bash
# Add to API
curl -X "POST" "https://mangrove-atlas-api.herokuapp.com/api/locations/import?reset=true" \
      -H "Content-Type: multipart/form-data" \
      -F "file=@locations_aoi-country-wdpa_20190924.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9707k    0     0  100 9707k      0   579k  0:00:16  0:00:16 --:--:--     0


In [0]:
# Get gee analysis output
# FIXME MAKE GEE OUTPUT FILE AUTOMATIC NAMED
import os
import subprocess
bdir = 'ee-export-tables'
b_file_path = 'countries_data_2019085_ee_export'
p = 'gs://' + os.environ.get('MA_BUCKET') + '/' + bdir + '/' + b_file_path + '.geojson'
subprocess.run(['gsutil', 'cp', p, '.'])

CompletedProcess(args=['gsutil', 'cp', 'gs://mangrove_atlas/ee-export-tables/countries_data_2019085_ee_export.geojson', '.'], returncode=0)

In [0]:
# Convert from geojson to a flat CSV structure
import json
import pandas as pd

data=json.load(open(b_file_path + '.geojson'))['features']

df_list = []
for f in data:
  geometry = f.get('geometry')
  #print(geometry)
  ids = f.get('id')
  props = f.get('properties')
  chs = props['con_hotspot_summary_km2']
  del props['con_hotspot_summary_km2']
  #print(props)
  out = pd.DataFrame(props)
  out['id'] = ids
  out['geometry'] = json.dumps(geometry)
  out['con_hotspot_summary_km2'] = json.dumps(chs)
  df_list.append(out)
  
df = pd.concat(df_list)
df = df.rename(columns={"type": "location_type", "length_coast_m": "coast_length_m", "id": "location_id", \
                        'agb_mangrove_mgha-1': 'agb_mgha_1', 'hba_mangrove_m': 'hba_m', 'hmax_mangrove_m': 'hmax_m', \
                       'mangrove_gain_m2': 'gain_m2', 'mangrove_loss_m2': 'loss_m2', 'length_mangrove_m': 'length_m' \
                        ,'area_mangrove_m2': 'area_m2'})
#df
df = df[['gain_m2', 'loss_m2', 'length_m', 'area_m2', 'hmax_m', 'agb_mgha_1', 'hba_m', 'location_id', 'location_type', 'name', 'con_hotspot_summary_km2']]
df = df.reset_index()
#df
df = df.rename(columns={"index": "date"})
df
df['date'] = pd.to_datetime(df['date'], format='%Y', errors='coerce')
df.to_csv(b_file_path + '.csv', index=False, na_rep="NA", sep=";")

In [0]:
df[['location_id']].size == pd.unique(df['location_id']).size

False

In [0]:
# Send to GCS
import os
import subprocess
p_csv = b_file_path + '.csv'
p = 'gs://' + os.environ.get('MA_BUCKET') + '/' + bdir
subprocess.run(['gsutil', 'cp', p_csv, p])


CompletedProcess(args=['gsutil', 'cp', 'countries_data_2019085_ee_export.csv', 'gs://mangrove_atlas/ee-export-tables'], returncode=0)

In [0]:
# View CSV
import pandas as pd
pd.read_csv(b_file_path + '.csv', sep=";")


,date,gain_m2,loss_m2,length_m,area_m2,hmax_m,agb_mgha_1,hba_m,location_id,location_type,name,con_hotspot_summary_km2
0,1996-01-01,0.000000e+00,0.000000e+00,65167.28,5.075309e+08,NaN,NaN,NaN,1_1_00000000000000000000,aoi,Rufiji,"{""Monitoring Advised"": 31.2962, ""Requires Moni..."
1,2000-01-01,NaN,NaN,NaN,NaN,19.92,260.21,12.62,1_1_00000000000000000000,aoi,Rufiji,"{""Monitoring Advised"": 31.2962, ""Requires Moni..."
2,2007-01-01,5.985117e+06,1.404493e+07,64694.98,4.994764e+08,NaN,NaN,NaN,1_1_00000000000000000000,aoi,Rufiji,"{""Monitoring Advised"": 31.2962, ""Requires Moni..."
3,2008-01-01,5.131899e+06,4.469590e+06,64148.45,5.001376e+08,NaN,NaN,NaN,1_1_00000000000000000000,aoi,Rufiji,"{""Monitoring Advised"": 31.2962, ""Requires Moni..."
4,2009-01-01,3.208974e+06,4.309829e+06,63901.10,4.990318e+08,NaN,NaN,NaN,1_1_00000000000000000000,aoi,Rufiji,"{""Monitoring Advised"": 31.2962, ""Requires Moni..."
5,2010-01-01,4.316326e+06,5.752221e+06,64280.28,4.975992e+08,NaN,NaN,NaN,1_1_00000000000000000000,aoi,Rufiji,"{""Monitoring Advised"": 31.2962, ""Requires Moni..."
6,2015-01-01,1.313068e+07,1.132364e+07,65776.31,4.994017e+08,NaN,NaN,NaN,1_1_00000000000000000000,aoi,Rufiji,"{""Monitoring Advised"": 31.2962, ""Requires Moni..."
7,2016-01-01,1.041104e+07,1.104505e+07,65467.70,4.987754e+08,NaN,NaN,NaN,1_1_00000000000000000000,aoi,Rufiji,"{""Monitoring Advised"": 31.2962, ""Requires Moni..."
8,1996-01-01,0.000000e+00,0.000000e+00,74792.05,5.777505e+08,NaN,NaN,NaN,1_2_00000000000000000000,aoi,Saloum,"{""Monitoring Advised"": 591.08, ""Requires Monit..."
9,2000-01-01,NaN,NaN,NaN,NaN,5.96,28.10,3.78,1_2_00000000000000000000,aoi,Saloum,"{""Monitoring Advised"": 591.08, ""Requires Monit..."


# Process environmental pressures

+ conservation hotspots


In [0]:
# Get data files
import os
import subprocess
b_file_path = 'environmental-pressures/OneDrive_2019-05-23.zip'
p = 'gs://' + os.environ.get('MA_BUCKET') + '/' + b_file_path
lp = os.environ.get('MA_BUCKET') + '/' + b_file_path
subprocess.run(['gsutil', 'cp', p, lp])

CompletedProcess(args=['gsutil', 'cp', 'gs://mangrove_atlas/environmental-pressures/OneDrive_2019-05-23.zip', 'mangrove_atlas/environmental-pressures/OneDrive_2019-05-23.zip'], returncode=0)

In [0]:
ls mangrove_atlas/environmental-pressures/shapefiles

'Hotspots for conservation'/


In [0]:
%%bash
# Process zipped shapefiles
# Clean and simplfy (parameters set above in section setup)
# Convert to geojson
# change mapshaper settings below, see https://github.com/mbloch/mapshaper/wiki/Command-Reference

# Set working directories
df=$MA_BUCKET/environmental-pressures
sf=$MA_BUCKET/environmental-pressures/shapefiles
pf=$MA_BUCKET/environmental-pressures/processed

# Create directories (if needed)
mkdir $sf
mkdir $pf
mkdir ${pf}/geojson

# Extract original ZIP to $sf
arr=($df/*.zip)
for ((i=0; i<${#arr[@]}; i++)); do
  z="${arr[$i]}"
  echo $z
  unzip -o $z -d $sf
  f="${z%%.*}"
done

# Clean and simplify original SHP
s=$sf/'Hotspots for conservation'/*.shp
echo $s
mapshaper-xl \
  -i snap files="${s}" \
  -simplify weighted interval=$SIMP_WEIGHT_INT \
  -clean min-gap-area=$MIN_AREA \
  -filter-islands min-area=$MIN_AREA \
  -filter-slivers min-area=$MIN_AREA \
  -clean min-gap-area=$MIN_AREA \
  -o $pf format=shapefile precision=0.000001 force

# Export processed SHP as geojson to $pf/geojson
sp=$pf/*.shp
echo $sp
mapshaper-xl \
  -i files="${sp}" \
  -o ${pf}/geojson format=geojson precision=0.000001 force

mangrove_atlas/environmental-pressures/OneDrive_2019-05-23.zip
Archive:  mangrove_atlas/environmental-pressures/OneDrive_2019-05-23.zip
 extracting: mangrove_atlas/environmental-pressures/shapefiles/Hotspots for conservation/cons_hotspots.cpg  
 extracting: mangrove_atlas/environmental-pressures/shapefiles/Hotspots for conservation/cons_hotspots.dbf  
 extracting: mangrove_atlas/environmental-pressures/shapefiles/Hotspots for conservation/cons_hotspots.prj  
 extracting: mangrove_atlas/environmental-pressures/shapefiles/Hotspots for conservation/cons_hotspots.qpj  
 extracting: mangrove_atlas/environmental-pressures/shapefiles/Hotspots for conservation/cons_hotspots.shp  
 extracting: mangrove_atlas/environmental-pressures/shapefiles/Hotspots for conservation/cons_hotspots.shx  
 extracting: mangrove_atlas/environmental-pressures/shapefiles/Hotspots for conservation/LayersInfo.docx  
mangrove_atlas/environmental-pressures/shapefiles/Hotspots for conservation/*.shp
mangrove_atlas/enviro

mkdir: cannot create directory ‘mangrove_atlas/environmental-pressures/shapefiles’: File exists
mkdir: cannot create directory ‘mangrove_atlas/environmental-pressures/processed’: File exists
mkdir: cannot create directory ‘mangrove_atlas/environmental-pressures/processed/geojson’: File exists
Allocating 8000 megabytes of heap memory
[i] Snapped 42 points
[simplify] Repaired 1,249 intersections; 729 intersections could not be repaired
[clean] Retained 367 of 394 features
[filter-islands] Removed 139 islands
[filter-slivers] Removed 83 slivers
[clean] Retained 367 of 367 features
[o] Wrote mangrove_atlas/environmental-pressures/processed/cons_hotspots.shp
[o] Wrote mangrove_atlas/environmental-pressures/processed/cons_hotspots.shx
[o] Wrote mangrove_atlas/environmental-pressures/processed/cons_hotspots.dbf
[o] Wrote mangrove_atlas/environmental-pressures/processed/cons_hotspots.prj
Allocating 8000 megabytes of heap memory
[o] Wrote mangrove_atlas/environmental-pressures/processed/geojson

In [0]:
# Zip processed shape files
%%bash
pf=$MA_BUCKET/environmental-pressures/processed
fl=(${pf}/*.shp)
for ((i=0; i<${#fl[@]}; i++)); do
  z="${fl[$i]}"
  bn="${z%%.*}"
  zip $bn.zip $bn.*
  
done

  adding: mangrove_atlas/environmental-pressures/processed/cons_hotspots.dbf (deflated 84%)
  adding: mangrove_atlas/environmental-pressures/processed/cons_hotspots.prj (deflated 15%)
  adding: mangrove_atlas/environmental-pressures/processed/cons_hotspots.shp (deflated 62%)
  adding: mangrove_atlas/environmental-pressures/processed/cons_hotspots.shx (deflated 27%)


In [0]:
%%bash
# Upload to GCS bucket
pf=$MA_BUCKET/environmental-pressures/processed
gsutil -m cp ${pf}/*.zip gs://${pf}/
gsutil -m cp -r ${pf}/geojson gs://${pf}/

Copying file://mangrove_atlas/environmental-pressures/processed/cons_hotspots.zip [Content-Type=application/zip]...
-
Operation completed over 1 objects/1.7 MiB.                                      
Copying file://mangrove_atlas/environmental-pressures/processed/geojson/cons_hotspots.json [Content-Type=application/json]...
-
Operation completed over 1 objects/5.9 MiB.                                      


In [0]:
# Add zipped SHPs as GEE assets
# FIXME use the manifest upload method?
# add METADATA!!
import os
file_path="gs://{}/environmental-pressures/processed/*.zip".format(os.environ.get('MA_BUCKET'))
asset_path="projects/global-mangrove-watch/environmental-pressures"
time_start="2010-01-01T00:00:00"

# Upload latest assets
gee_upload_zips_to_tables(file_path, asset_path, time_start, force=True, append_txt="_simplified", properties={})

Found 2 files
Processing cons_hotspots
earthengine --no-use_cloud_api upload table --asset_id=projects/global-mangrove-watch/environmental-pressures/cons_hotspots_simplified --force --time_start=2010-01-01T00:00:00   gs://mangrove_atlas/environmental-pressures/processed/cons_hotspots.zip
Task created
Processing 
earthengine --no-use_cloud_api upload table --asset_id=projects/global-mangrove-watch/environmental-pressures/_simplified --force --time_start=2010-01-01T00:00:00   
Task failed
Finished upload


In [0]:
# Upload to mapbox
# Get processed boundaries data
!rm -r $MA_BUCKET/environmental-pressures/processed/geojson
!gsutil -m cp -r gs://$MA_BUCKET/environmental-pressures/processed/geojson $MA_BUCKET/environmental-pressures/processed
# Rename GEE exports  
#!mv ma-data/boundaries/processed/geojson/selected_EEZ_land_v2_201410_simplified.json ma-data/boundaries/processed/geojson/selected_EEZ_land_v2_201410.json
#!mv ma-data/boundaries/processed/geojson/selected_WDPA_polygons_ee_export.json ma-data/boundaries/processed/geojson/selected_WDPA_polygons.json  
    
import glob
path = '{}/environmental-pressures/processed/geojson'.format(os.environ.get('MA_BUCKET'))
files = [f for f in glob.glob(path + "**/*.json", recursive=True)]
for f in files:
  upload_to_mapbox(f, os.path.splitext(os.path.basename(f))[0])

Copying gs://mangrove_atlas/environmental-pressures/processed/geojson/cons_hotspots.json...
/ [1/1 files][  5.9 MiB/  5.9 MiB] 100% Done                                    
Operation completed over 1 objects/5.9 MiB.                                      
{'message': 'Not Found'}


# Process mangrove alerts